# 실습 1일차
---

## 1. 문제 정의


### 한국프로야구 관중현황 데이터 분석
 - 요일별(평일, 금요일, 토요일) 관중 수는 어떤 차이가 있을까?
 - 가장 인기가 많은 팀은 어떤 팀일까?
 - 기온과 구장 별 관중 수는 어떤 관계가 있을까?
 - 강수여부와 구장 별 관중 수는 어떤 관계가 있을까?
 
---

## 2. 데이터 수집

### 야구 관중 현황 데이터 수집하기

Google "야구 관중 데이터" -> KBO 홈페이지의 관중현황 페이지

https://www.koreabaseball.com/History/Crowd/GraphDaily.aspx

2019년은 아직 시즌 진행중이므로, 2017년과 2018년 데이터를 수집


#### <실습>
 - 2017년, 2018년 일자별 관중현황을 DataFrame에 불러와 보자
 - 힌트 : pd.read_html(url)
 - 의도한 대로 데이터가 수집되는가?

In [121]:
import pandas as pd

df_baseball = pd.read_html('https://www.koreabaseball.com/History/Crowd/GraphDaily.aspx')[0]
print(df_baseball)

[                                            날짜  \
0                                   2019/03/23   
1                                   2019/03/23   
2                                   2019/03/23   
3                                   2019/03/23   
4                                   2019/03/23   
5                                   2019/03/24   
6                                   2019/03/24   
7                                   2019/03/24   
8                                   2019/03/24   
9                                   2019/03/24   
10                                  2019/03/26   
11                                  2019/03/26   
12                                  2019/03/26   
13                                  2019/03/26   
14                                  2019/03/26   
15                                  2019/03/27   
16                                  2019/03/27   
17                                  2019/03/27   
18                                  2019/03/27   

검색항목에서 '시즌' 항목을 2017년으로 바꿔도, URL에는 반영이 되지 않는다!

#### 첫 번째 대안 : 웹 크롤링

 - 웹 브라우저에서 버튼을 누르거나, 페이지를 이동하는 대부분의 행위는, 해당 웹페이지 서버에 'Request'로 전달된다.
 - 'Postman' 앱 및 'Postman Interceptor' 크롬 애드온을 통해 우리가 보내는 'Request'를 파이썬 코드로 변환할 수 있다. 
 - 수강신청이나 티켓팅 등에도 유용하게 사용 가능
 
 ![title](img/postman_history.PNG)
 
 ![title](img/postman_body.PNG)
 
 ![title](img/postman_code.PNG)

In [122]:
import http.client
import json
conn = http.client.HTTPSConnection("www.koreabaseball.com")

payload = "ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24ScriptManager1=ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24udpRecord%7Cctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24btnSearch&__EVENTTARGET=&__EVENTARGUMENT=&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlSeason=2018&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlMonth=0&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlTeam=&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlHomeAway=&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlStadium=&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24ddlDayOfWeek=0&__VIEWSTATE=%2FwEPDwULLTE5NjgwMjcwNjYPZBYCZg9kFgJmD2QWAmYPZBYCAgMPZBYCZg9kFgICAQ9kFgICBQ9kFgICAQ9kFgJmD2QWFgIBDxBkEBUECeyLnOymjOuzhAQyMDE3BDIwMTgEMjAxORUEBDIwMTkEMjAxNwQyMDE4BDIwMTkUKwMEZ2dnZxYBAgJkAgMPEGQQFQsG7JuU67OEBTAz7JuUBTA07JuUBTA17JuUBTA27JuUBTA37JuUBTA47JuUBTA57JuUBTEw7JuUBTEx7JuUBTEy7JuUFQsBMAEzATQBNQE2ATcBOAE5AjEwAjExAjEyFCsDC2dnZ2dnZ2dnZ2dnFgFmZAIFDxBkEBULBu2MgOuzhAbrkZDsgrAG66Gv642wBuyCvOyEsQbtgqTsm4AG7ZWc7ZmUA0tJQQJLVAJMRwJOQwJTSxULAAJPQgJMVAJTUwJXTwJISAJIVAJLVAJMRwJOQwJTSxQrAwtnZ2dnZ2dnZ2dnZxYBZmQCBw8QZBAVAw3tmYgv67Cp66y467OEA%2B2ZiAbrsKnrrLgVAwABQgFUFCsDA2dnZxYBZmQCCQ8QZBAVDwnqtazsnqXrs4QG6rOg7LKZBuq0keyjvAbrjIDqtawG64yA7KCEBuusuO2VmQbsgqzsp4EG7IOB64%2BZBuyImOybkAbsmrjsgrAK7J207LKcKExHKQbsnqDsi6QG7LC97JuQBuyyreyjvAbtj6ztla0VDwACR0MCS0MCREsCREoCTUgCU0oCU0QCU1cCVUwCRUwCSlMCQ1cCQ0oCUEgUKwMPZ2dnZ2dnZ2dnZ2dnZ2dnFgFmZAILDxBkEBUICeyalOydvOuzhAPtmZQD7IiYA%2BuqqQPquIgD7YagA%2BydvAPsm5QVCAEwATIBMwE0ATUBNgE3ATEUKwMIZ2dnZ2dnZ2cWAWZkAg8PDxYCHgRUZXh0BVzqsr3quLAg6rSA7KSR7ZiE7ZmpKOyblDrsoITssrQsIO2MgDrsoITssrQsIO2ZiC%2FrsKnrrLg67KCE7LK0LCDqtazsnqU67KCE7LK0LCDsmpTsnbw67KCE7LK0KWRkAhEPDxYCHwAFAzcyMGRkAhMPDxYCHwAFBjExLDIxNGRkAhUPDxYCHwAFCTgsMDczLDc0MmRkAhcPFgIeC18hSXRlbUNvdW50AtAFFqALZg9kFgJmDxUGCjIwMTgvMDMvMjQD7YagBuuEpeyEvAbtlZztmZQG6rOg7LKZBjE1LDA1NWQCAQ9kFgJmDxUGCjIwMTgvMDMvMjQD7YagA0tJQQJLVAbqtJHso7wGMjAsNTAwZAICD2QWAmYPFQYKMjAxOC8wMy8yNAPthqACTkMCTEcG66eI7IKwBjExLDAwMGQCAw9kFgJmDxUGCjIwMTgvMDMvMjQD7YagAlNLBuuhr%2BuNsAbrrLjtlZkGMjUsMDAwZAIED2QWAmYPFQYKMjAxOC8wMy8yNAPthqAG65GQ7IKwBuyCvOyEsQbsnqDsi6QGMjUsMDAwZAIFD2QWAmYPFQYKMjAxOC8wMy8yNQPsnbwG64Sl7IS8Bu2VnO2ZlAbqs6DsspkGMTMsMDEwZAIGD2QWAmYPFQYKMjAxOC8wMy8yNQPsnbwDS0lBAktUBuq0keyjvAYyMCw1MDBkAgcPZBYCZg8VBgoyMDE4LzAzLzI1A%2BydvAJOQwJMRwbrp4jsgrAGMTEsMDAwZAIID2QWAmYPFQYKMjAxOC8wMy8yNQPsnbwCU0sG66Gv642wBuusuO2VmQYyMiw3NjVkAgkPZBYCZg8VBgoyMDE4LzAzLzI1A%2BydvAbrkZDsgrAG7IK87ISxBuyeoOyLpAYyMCwyNDBkAgoPZBYCZg8VBgoyMDE4LzAzLzI3A%2B2ZlAJOQwbtlZztmZQG66eI7IKwBTUsMDM3ZAILD2QWAmYPFQYKMjAxOC8wMy8yNwPtmZQCU0sCS1QG66y47ZWZBTYsMDY3ZAIMD2QWAmYPFQYKMjAxOC8wMy8yNwPtmZQG64Sl7IS8AkxHBuqzoOyymQU0LDQwMWQCDQ9kFgJmDxUGCjIwMTgvMDMvMjcD7ZmUBuuRkOyCsAbroa%2FrjbAG7J6g7IukBjEwLDM5MWQCDg9kFgJmDxUGCjIwMTgvMDMvMjcD7ZmUA0tJQQbsgrzshLEG6rSR7KO8BTYsODkyZAIPD2QWAmYPFQYKMjAxOC8wMy8yOAPsiJgCTkMG7ZWc7ZmUBuuniOyCsAU1LDAwM2QCEA9kFgJmDxUGCjIwMTgvMDMvMjgD7IiYAlNLAktUBuusuO2VmQU1LDcwMWQCEQ9kFgJmDxUGCjIwMTgvMDMvMjgD7IiYBuuEpeyEvAJMRwbqs6DsspkFNCw0MzdkAhIPZBYCZg8VBgoyMDE4LzAzLzI4A%2ByImAbrkZDsgrAG66Gv642wBuyeoOyLpAYxMCwzODJkAhMPZBYCZg8VBgoyMDE4LzAzLzI4A%2ByImANLSUEG7IK87ISxBuq0keyjvAU4LDM0N2QCFA9kFgJmDxUGCjIwMTgvMDMvMjkD66qpAk5DBu2VnO2ZlAbrp4jsgrAFNCw4NzJkAhUPZBYCZg8VBgoyMDE4LzAzLzI5A%2BuqqQJTSwJLVAbrrLjtlZkFNSw5MzZkAhYPZBYCZg8VBgoyMDE4LzAzLzI5A%2BuqqQbrhKXshLwCTEcG6rOg7LKZBTQsMjE5ZAIXD2QWAmYPFQYKMjAxOC8wMy8yOQPrqqkG65GQ7IKwBuuhr%2BuNsAbsnqDsi6QGMTAsOTU2ZAIYD2QWAmYPFQYKMjAxOC8wMy8yOQPrqqkDS0lBBuyCvOyEsQbqtJHso7wFNyw4MzlkAhkPZBYCZg8VBgoyMDE4LzAzLzMwA%2Bq4iAJMRwNLSUEG7J6g7IukBjIyLDY2N2QCGg9kFgJmDxUGCjIwMTgvMDMvMzAD6riIBuuhr%2BuNsAJOQwbsgqzsp4EGMjEsMTI0ZAIbD2QWAmYPFQYKMjAxOC8wMy8zMAPquIgCS1QG65GQ7IKwBuyImOybkAYxMyw1NjJkAhwPZBYCZg8VBgoyMDE4LzAzLzMwA%2Bq4iAbtlZztmZQCU0sG64yA7KCEBjEwLDM1NGQCHQ9kFgJmDxUGCjIwMTgvMDMvMzAD6riIBuyCvOyEsQbrhKXshLwG64yA6rWsBjExLDY4M2QCHg9kFgJmDxUGCjIwMTgvMDMvMzED7YagAkxHA0tJQQbsnqDsi6QGMjUsMDAwZAIfD2QWAmYPFQYKMjAxOC8wMy8zMQPthqAG66Gv642wAk5DBuyCrOyngQYyNSwwMDBkAiAPZBYCZg8VBgoyMDE4LzAzLzMxA%2B2GoAJLVAbrkZDsgrAG7IiY7JuQBjE2LDg1MmQCIQ9kFgJmDxUGCjIwMTgvMDMvMzED7YagBu2VnO2ZlAJTSwbrjIDsoIQGMTMsMDAwZAIiD2QWAmYPFQYKMjAxOC8wMy8zMQPthqAG7IK87ISxBuuEpeyEvAbrjIDqtawGMTYsNTcyZAIjD2QWAmYPFQYKMjAxOC8wNC8wMQPsnbwCTEcDS0lBBuyeoOyLpAYyMyw4MTVkAiQPZBYCZg8VBgoyMDE4LzA0LzAxA%2BydvAbroa%2FrjbACTkMG7IKs7KeBBjE5LDQzMmQCJQ9kFgJmDxUGCjIwMTgvMDQvMDED7J28AktUBuuRkOyCsAbsiJjsm5AGMTYsMTM3ZAImD2QWAmYPFQYKMjAxOC8wNC8wMQPsnbwG7ZWc7ZmUAlNLBuuMgOyghAU5LDUyOGQCJw9kFgJmDxUGCjIwMTgvMDQvMDED7J28BuyCvOyEsQbrhKXshLwG64yA6rWsBjE0LDMzNmQCKA9kFgJmDxUGCjIwMTgvMDQvMDMD7ZmUAlNLA0tJQQbrrLjtlZkGMTUsMDUyZAIpD2QWAmYPFQYKMjAxOC8wNC8wMwPtmZQG64Sl7IS8AktUBuqzoOyymQUxLDgxNGQCKg9kFgJmDxUGCjIwMTgvMDQvMDMD7ZmUBuuRkOyCsAJMRwbsnqDsi6QGMTMsNTU3ZAIrD2QWAmYPFQYKMjAxOC8wNC8wMwPtmZQG7ZWc7ZmUBuuhr%2BuNsAbrjIDsoIQFNSw3NDBkAiwPZBYCZg8VBgoyMDE4LzA0LzAzA%2B2ZlAJOQwbsgrzshLEG66eI7IKwBTUsNjU1ZAItD2QWAmYPFQYKMjAxOC8wNC8wNAPsiJgCU0sDS0lBBuusuO2VmQYxNCw2MzlkAi4PZBYCZg8VBgoyMDE4LzA0LzA0A%2ByImAbrhKXshLwCS1QG6rOg7LKZBTIsMDgxZAIvD2QWAmYPFQYKMjAxOC8wNC8wNAPsiJgG65GQ7IKwAkxHBuyeoOyLpAYxMiw3NDJkAjAPZBYCZg8VBgoyMDE4LzA0LzA0A%2ByImAbtlZztmZQG66Gv642wBuuMgOyghAUzLDg5MWQCMQ9kFgJmDxUGCjIwMTgvMDQvMDQD7IiYAk5DBuyCvOyEsQbrp4jsgrAFMywxOTJkAjIPZBYCZg8VBgoyMDE4LzA0LzA1A%2BuqqQbrhKXshLwCS1QG6rOg7LKZBTIsNDg5ZAIzD2QWAmYPFQYKMjAxOC8wNC8wNQPrqqkCTkMG7IK87ISxBuuniOyCsAUzLDkwNGQCNA9kFgJmDxUGCjIwMTgvMDQvMDYD6riIBuuhr%2BuNsAJMRwbsgqzsp4EFOCw4MTRkAjUPZBYCZg8VBgoyMDE4LzA0LzA2A%2Bq4iANLSUEG64Sl7IS8Buq0keyjvAYxMCw2MTZkAjYPZBYCZg8VBgoyMDE4LzA0LzA3A%2B2GoAJLVAbtlZztmZQG7IiY7JuQBjE0LDc0NmQCNw9kFgJmDxUGCjIwMTgvMDQvMDcD7YagBuuhr%2BuNsAJMRwbsgqzsp4EGMTQsNDk4ZAI4D2QWAmYPFQYKMjAxOC8wNC8wNwPthqAG65GQ7IKwAk5DBuyeoOyLpAYxNiw1ODZkAjkPZBYCZg8VBgoyMDE4LzA0LzA3A%2B2GoAJTSwbsgrzshLEG66y47ZWZBjE3LDU4MWQCOg9kFgJmDxUGCjIwMTgvMDQvMDcD7YagA0tJQQbrhKXshLwG6rSR7KO8BjE2LDY2MWQCOw9kFgJmDxUGCjIwMTgvMDQvMDgD7J28AktUBu2VnO2ZlAbsiJjsm5AGMTIsODQwZAI8D2QWAmYPFQYKMjAxOC8wNC8wOAPsnbwG66Gv642wAkxHBuyCrOyngQYxMywwOTNkAj0PZBYCZg8VBgoyMDE4LzA0LzA4A%2BydvAbrkZDsgrACTkMG7J6g7IukBjEzLDg3N2QCPg9kFgJmDxUGCjIwMTgvMDQvMDgD7J28AlNLBuyCvOyEsQbrrLjtlZkGMTUsMTQ0ZAI%2FD2QWAmYPFQYKMjAxOC8wNC8wOAPsnbwDS0lBBuuEpeyEvAbqtJHso7wGMTYsMTMzZAJAD2QWAmYPFQYKMjAxOC8wNC8xMAPtmZQG7ZWc7ZmUA0tJQQbrjIDsoIQFNiw1MzlkAkEPZBYCZg8VBgoyMDE4LzA0LzEwA%2B2ZlAJOQwJLVAbrp4jsgrAFNiwwMTBkAkIPZBYCZg8VBgoyMDE4LzA0LzEwA%2B2ZlAbsgrzshLEG65GQ7IKwBuuMgOq1rAU2LDMyM2QCQw9kFgJmDxUGCjIwMTgvMDQvMTAD7ZmUAkxHAlNLBuyeoOyLpAU4LDA2OWQCRA9kFgJmDxUGCjIwMTgvMDQvMTAD7ZmUBuuhr%2BuNsAbrhKXshLwG7Jq47IKwBTUsOTAyZAJFD2QWAmYPFQYKMjAxOC8wNC8xMQPsiJgG7ZWc7ZmUA0tJQQbrjIDsoIQFNyw1NDVkAkYPZBYCZg8VBgoyMDE4LzA0LzExA%2ByImAJOQwJLVAbrp4jsgrAFNSwzOTNkAkcPZBYCZg8VBgoyMDE4LzA0LzExA%2ByImAbsgrzshLEG65GQ7IKwBuuMgOq1rAU3LDczNmQCSA9kFgJmDxUGCjIwMTgvMDQvMTED7IiYAkxHAlNLBuyeoOyLpAU4LDk0MGQCSQ9kFgJmDxUGCjIwMTgvMDQvMTED7IiYBuuhr%2BuNsAbrhKXshLwG7Jq47IKwBTUsOTkxZAJKD2QWAmYPFQYKMjAxOC8wNC8xMgPrqqkG7ZWc7ZmUA0tJQQbrjIDsoIQFOCw1NzdkAksPZBYCZg8VBgoyMDE4LzA0LzEyA%2BuqqQJOQwJLVAbrp4jsgrAFNSw4NTVkAkwPZBYCZg8VBgoyMDE4LzA0LzEyA%2BuqqQbsgrzshLEG65GQ7IKwBuuMgOq1rAU2LDI4M2QCTQ9kFgJmDxUGCjIwMTgvMDQvMTID66qpAkxHAlNLBuyeoOyLpAU5LDgyNWQCTg9kFgJmDxUGCjIwMTgvMDQvMTID66qpBuuhr%2BuNsAbrhKXshLwG7Jq47IKwBTcsMjUyZAJPD2QWAmYPFQYKMjAxOC8wNC8xMwPquIgCTEcCS1QG7J6g7IukBjEwLDg5OWQCUA9kFgJmDxUGCjIwMTgvMDQvMTMD6riIA0tJQQbroa%2FrjbAG6rSR7KO8BjE0LDE1OWQCUQ9kFgJmDxUGCjIwMTgvMDQvMTMD6riIAlNLAk5DBuusuO2VmQU3LDk0NWQCUg9kFgJmDxUGCjIwMTgvMDQvMTMD6riIBuuEpeyEvAbrkZDsgrAG6rOg7LKZBTYsMDMxZAJTD2QWAmYPFQYKMjAxOC8wNC8xMwPquIgG7ZWc7ZmUBuyCvOyEsQbrjIDsoIQFOCwwNzJkAlQPZBYCZg8VBgoyMDE4LzA0LzE0A%2B2GoAJMRwJLVAbsnqDsi6QGMTIsODYyZAJVD2QWAmYPFQYKMjAxOC8wNC8xNAPthqACU0sCTkMG66y47ZWZBTksOTM5ZAJWD2QWAmYPFQYKMjAxOC8wNC8xNAPthqAG64Sl7IS8BuuRkOyCsAbqs6DsspkGMTIsNDgwZAJXD2QWAmYPFQYKMjAxOC8wNC8xNAPthqAG7ZWc7ZmUBuyCvOyEsQbrjIDsoIQFOCw3ODNkAlgPZBYCZg8VBgoyMDE4LzA0LzE1A%2BydvAJMRwJLVAbsnqDsi6QGMTUsMzM2ZAJZD2QWAmYPFQYKMjAxOC8wNC8xNQPsnbwCU0sCTkMG66y47ZWZBjE1LDQ3MWQCWg9kFgJmDxUGCjIwMTgvMDQvMTUD7J28BuuEpeyEvAbrkZDsgrAG6rOg7LKZBTksNjI5ZAJbD2QWAmYPFQYKMjAxOC8wNC8xNQPsnbwG7ZWc7ZmUBuyCvOyEsQbrjIDsoIQGMTMsMDAwZAJcD2QWAmYPFQYKMjAxOC8wNC8xNwPtmZQG65GQ7IKwBu2VnO2ZlAbsnqDsi6QGMTYsMDczZAJdD2QWAmYPFQYKMjAxOC8wNC8xNwPtmZQDS0lBAkxHBuq0keyjvAU4LDA1OGQCXg9kFgJmDxUGCjIwMTgvMDQvMTcD7ZmUBuuEpeyEvAJOQwbqs6DsspkFMiwwMjhkAl8PZBYCZg8VBgoyMDE4LzA0LzE3A%2B2ZlAJLVAJTSwbsiJjsm5AFNCwyMzdkAmAPZBYCZg8VBgoyMDE4LzA0LzE3A%2B2ZlAbroa%2FrjbAG7IK87ISxBuyCrOyngQU5LDIxM2QCYQ9kFgJmDxUGCjIwMTgvMDQvMTgD7IiYBuuRkOyCsAbtlZztmZQG7J6g7IukBjE2LDQzN2QCYg9kFgJmDxUGCjIwMTgvMDQvMTgD7IiYA0tJQQJMRwbqtJHso7wFOCw1MjhkAmMPZBYCZg8VBgoyMDE4LzA0LzE4A%2ByImAbrhKXshLwCTkMG6rOg7LKZBTIsMzQ0ZAJkD2QWAmYPFQYKMjAxOC8wNC8xOAPsiJgCS1QCU0sG7IiY7JuQBTQsMzEyZAJlD2QWAmYPFQYKMjAxOC8wNC8xOAPsiJgG66Gv642wBuyCvOyEsQbsgqzsp4EFOCw0MjhkAmYPZBYCZg8VBgoyMDE4LzA0LzE5A%2BuqqQbrkZDsgrAG7ZWc7ZmUBuyeoOyLpAYxNiwxMDRkAmcPZBYCZg8VBgoyMDE4LzA0LzE5A%2BuqqQNLSUECTEcG6rSR7KO8BjEwLDcwMGQCaA9kFgJmDxUGCjIwMTgvMDQvMTkD66qpBuuEpeyEvAJOQwbqs6DsspkFMiw0NTNkAmkPZBYCZg8VBgoyMDE4LzA0LzE5A%2BuqqQJLVAJTSwbsiJjsm5AFNCw2NDBkAmoPZBYCZg8VBgoyMDE4LzA0LzE5A%2BuqqQbroa%2FrjbAG7IK87ISxBuyCrOyngQU5LDM3OWQCaw9kFgJmDxUGCjIwMTgvMDQvMjAD6riIBuuRkOyCsANLSUEG7J6g7IukBjIzLDIxNGQCbA9kFgJmDxUGCjIwMTgvMDQvMjAD6riIBuyCvOyEsQJLVAbrjIDqtawFNyw0NjZkAm0PZBYCZg8VBgoyMDE4LzA0LzIwA%2Bq4iAJOQwJMRwbrp4jsgrAFNywzNTdkAm4PZBYCZg8VBgoyMDE4LzA0LzIwA%2Bq4iAbroa%2FrjbACU0sG7IKs7KeBBTgsNjQ3ZAJvD2QWAmYPFQYKMjAxOC8wNC8yMAPquIgG7ZWc7ZmUBuuEpeyEvAbrjIDsoIQFOSwyNDZkAnAPZBYCZg8VBgoyMDE4LzA0LzIxA%2B2GoAbrkZDsgrADS0lBBuyeoOyLpAYyNSwwMDBkAnEPZBYCZg8VBgoyMDE4LzA0LzIxA%2B2GoAbsgrzshLECS1QG64yA6rWsBjE2LDQ1NGQCcg9kFgJmDxUGCjIwMTgvMDQvMjED7YagAk5DAkxHBuuniOyCsAU4LDc2N2QCcw9kFgJmDxUGCjIwMTgvMDQvMjED7YagBuuhr%2BuNsAJTSwbsgqzsp4EGMTcsOTExZAJ0D2QWAmYPFQYKMjAxOC8wNC8yMQPthqAG7ZWc7ZmUBuuEpeyEvAbrjIDsoIQGMTMsMDAwZAJ1D2QWAmYPFQYKMjAxOC8wNC8yMgPsnbwG65GQ7IKwA0tJQQbsnqDsi6QGMjIsMTc3ZAJ2D2QWAmYPFQYKMjAxOC8wNC8yMgPsnbwG7IK87ISxAktUBuuMgOq1rAU5LDAyNmQCdw9kFgJmDxUGCjIwMTgvMDQvMjID7J28Ak5DAkxHBuuniOyCsAU2LDI0MmQCeA9kFgJmDxUGCjIwMTgvMDQvMjID7J28Buuhr%2BuNsAJTSwbsgqzsp4EGMTEsMzY0ZAJ5D2QWAmYPFQYKMjAxOC8wNC8yMgPsnbwG7ZWc7ZmUBuuEpeyEvAbrjIDsoIQFNiw3NDNkAnoPZBYCZg8VBgoyMDE4LzA0LzI0A%2B2ZlAJLVAbroa%2FrjbAG7IiY7JuQBTUsNDI5ZAJ7D2QWAmYPFQYKMjAxOC8wNC8yNAPtmZQG7IK87ISxAk5DBuuMgOq1rAUzLDIxNmQCfA9kFgJmDxUGCjIwMTgvMDQvMjQD7ZmUAlNLBuuRkOyCsAbrrLjtlZkFNyw5MjlkAn0PZBYCZg8VBgoyMDE4LzA0LzI0A%2B2ZlAJMRwbrhKXshLwG7J6g7IukBTgsNTMxZAJ%2BD2QWAmYPFQYKMjAxOC8wNC8yNQPsiJgDS0lBBu2VnO2ZlAbqtJHso7wGMTAsNDIwZAJ%2FD2QWAmYPFQYKMjAxOC8wNC8yNQPsiJgCS1QG66Gv642wBuyImOybkAU2LDkyMmQCgAEPZBYCZg8VBgoyMDE4LzA0LzI1A%2ByImAbsgrzshLECTkMG64yA6rWsBTUsMzcxZAKBAQ9kFgJmDxUGCjIwMTgvMDQvMjUD7IiYAlNLBuuRkOyCsAbrrLjtlZkGMTQsMjAxZAKCAQ9kFgJmDxUGCjIwMTgvMDQvMjUD7IiYAkxHBuuEpeyEvAbsnqDsi6QGMTAsNzQ0ZAKDAQ9kFgJmDxUGCjIwMTgvMDQvMjYD66qpA0tJQQbtlZztmZQG6rSR7KO8BjEyLDA1NWQChAEPZBYCZg8VBgoyMDE4LzA0LzI2A%2BuqqQJLVAbroa%2FrjbAG7IiY7JuQBTcsMDIzZAKFAQ9kFgJmDxUGCjIwMTgvMDQvMjYD66qpBuyCvOyEsQJOQwbrjIDqtawFNyw3MDVkAoYBD2QWAmYPFQYKMjAxOC8wNC8yNgPrqqkCU0sG65GQ7IKwBuusuO2VmQYxNSwwMjBkAocBD2QWAmYPFQYKMjAxOC8wNC8yNgPrqqkCTEcG64Sl7IS8BuyeoOyLpAYxMSwwMDhkAogBD2QWAmYPFQYKMjAxOC8wNC8yNwPquIgG66Gv642wBu2VnO2ZlAbsgqzsp4EGMTQsODczZAKJAQ9kFgJmDxUGCjIwMTgvMDQvMjcD6riIAktUA0tJQQbsiJjsm5AGMTQsOTAxZAKKAQ9kFgJmDxUGCjIwMTgvMDQvMjcD6riIAk5DBuuRkOyCsAbrp4jsgrAFOCwzMzFkAosBD2QWAmYPFQYKMjAxOC8wNC8yNwPquIgG64Sl7IS8AlNLBuqzoOyymQUzLDI2MGQCjAEPZBYCZg8VBgoyMDE4LzA0LzI3A%2Bq4iAJMRwbsgrzshLEG7J6g7IukBjE1LDg4NWQCjQEPZBYCZg8VBgoyMDE4LzA0LzI4A%2B2GoAbroa%2FrjbAG7ZWc7ZmUBuyCrOyngQYyMywxMzlkAo4BD2QWAmYPFQYKMjAxOC8wNC8yOAPthqACS1QDS0lBBuyImOybkAYyMCw4MDBkAo8BD2QWAmYPFQYKMjAxOC8wNC8yOAPthqACTkMG65GQ7IKwBuuniOyCsAYxMCwwNThkApABD2QWAmYPFQYKMjAxOC8wNC8yOAPthqAG64Sl7IS8AlNLBuqzoOyymQU3LDE4NWQCkQEPZBYCZg8VBgoyMDE4LzA0LzI4A%2B2GoAJMRwbsgrzshLEG7J6g7IukBjIzLDM4MGQCkgEPZBYCZg8VBgoyMDE4LzA0LzI5A%2BydvAbroa%2FrjbAG7ZWc7ZmUBuyCrOyngQYxNyw2ODRkApMBD2QWAmYPFQYKMjAxOC8wNC8yOQPsnbwCS1QDS0lBBuyImOybkAYyMCw4MDBkApQBD2QWAmYPFQYKMjAxOC8wNC8yOQPsnbwCTkMG65GQ7IKwBuuniOyCsAU5LDA3NWQClQEPZBYCZg8VBgoyMDE4LzA0LzI5A%2BydvAbrhKXshLwCU0sG6rOg7LKZBTUsNzEyZAKWAQ9kFgJmDxUGCjIwMTgvMDQvMjkD7J28AkxHBuyCvOyEsQbsnqDsi6QGMjEsMDMxZAKXAQ9kFgJmDxUGCjIwMTgvMDUvMDED7ZmUBuuhr%2BuNsANLSUEG7IKs7KeBBjEyLDc1OGQCmAEPZBYCZg8VBgoyMDE4LzA1LzAxA%2B2ZlAbrkZDsgrACS1QG7J6g7IukBjEzLDQyMWQCmQEPZBYCZg8VBgoyMDE4LzA1LzAxA%2B2ZlAbtlZztmZQCTEcG64yA7KCEBjEyLDQ0OWQCmgEPZBYCZg8VBgoyMDE4LzA1LzAxA%2B2ZlAbsgrzshLECU0sG64yA6rWsBTYsNzY0ZAKbAQ9kFgJmDxUGCjIwMTgvMDUvMDED7ZmUAk5DBuuEpeyEvAbrp4jsgrAFNiw1MjRkApwBD2QWAmYPFQYKMjAxOC8wNS8wMgPsiJgG66Gv642wA0tJQQbsgqzsp4EFNyw1MTNkAp0BD2QWAmYPFQYKMjAxOC8wNS8wMgPsiJgG7ZWc7ZmUAkxHBuuMgOyghAU2LDE3MWQCngEPZBYCZg8VBgoyMDE4LzA1LzAyA%2ByImAbsgrzshLECU0sG64yA6rWsBTMsNTMxZAKfAQ9kFgJmDxUGCjIwMTgvMDUvMDID7IiYAk5DBuuEpeyEvAbrp4jsgrAFMywwOThkAqABD2QWAmYPFQYKMjAxOC8wNS8wMwPrqqkG66Gv642wA0tJQQbsgqzsp4EGMTIsMjY3ZAKhAQ9kFgJmDxUGCjIwMTgvMDUvMDMD66qpBuuRkOyCsAJLVAbsnqDsi6QGMTAsMTk2ZAKiAQ9kFgJmDxUGCjIwMTgvMDUvMDMD66qpBu2VnO2ZlAJMRwbrjIDsoIQFOSw2ODBkAqMBD2QWAmYPFQYKMjAxOC8wNS8wMwPrqqkG7IK87ISxAlNLBuuMgOq1rAU0LDc2MGQCpAEPZBYCZg8VBgoyMDE4LzA1LzAzA%2BuqqQJOQwbrhKXshLwG66eI7IKwBTQsMzM0ZAKlAQ9kFgJmDxUGCjIwMTgvMDUvMDQD6riIBuyCvOyEsQbtlZztmZQG64yA6rWsBjExLDA5NGQCpgEPZBYCZg8VBgoyMDE4LzA1LzA0A%2Bq4iAJTSwbroa%2FrjbAG66y47ZWZBjE3LDYyOGQCpwEPZBYCZg8VBgoyMDE4LzA1LzA0A%2Bq4iANLSUECTkMG6rSR7KO8BjE1LDg1MWQCqAEPZBYCZg8VBgoyMDE4LzA1LzA0A%2Bq4iAJMRwbrkZDsgrAG7J6g7IukBjIzLDA5OGQCqQEPZBYCZg8VBgoyMDE4LzA1LzA0A%2Bq4iAJLVAbrhKXshLwG7IiY7JuQBTUsNTcyZAKqAQ9kFgJmDxUGCjIwMTgvMDUvMDUD7YagBuyCvOyEsQbtlZztmZQG64yA6rWsBjIxLDI5MWQCqwEPZBYCZg8VBgoyMDE4LzA1LzA1A%2B2GoAJTSwbroa%2FrjbAG66y47ZWZBjI1LDAwMGQCrAEPZBYCZg8VBgoyMDE4LzA1LzA1A%2B2GoANLSUECTkMG6rSR7KO8BjIwLDUwMGQCrQEPZBYCZg8VBgoyMDE4LzA1LzA1A%2B2GoAJMRwbrkZDsgrAG7J6g7IukBjI1LDAwMGQCrgEPZBYCZg8VBgoyMDE4LzA1LzA1A%2B2GoAJLVAbrhKXshLwG7IiY7JuQBjE1LDA5MGQCrwEPZBYCZg8VBgoyMDE4LzA1LzA2A%2BydvANLSUECTkMG6rSR7KO8BjE2LDUzMmQCsAEPZBYCZg8VBgoyMDE4LzA1LzA2A%2BydvAJMRwbrkZDsgrAG7J6g7IukBjIxLDAxNGQCsQEPZBYCZg8VBgoyMDE4LzA1LzA4A%2B2ZlAbrhKXshLwG7ZWc7ZmUBuqzoOyymQU2LDg3NWQCsgEPZBYCZg8VBgoyMDE4LzA1LzA4A%2B2ZlAJMRwbroa%2FrjbAG7J6g7IukBjEzLDQ0OGQCswEPZBYCZg8VBgoyMDE4LzA1LzA4A%2B2ZlANLSUEG65GQ7IKwBuq0keyjvAU4LDczNGQCtAEPZBYCZg8VBgoyMDE4LzA1LzA4A%2B2ZlAJOQwJTSwbrp4jsgrAFNCwwNjhkArUBD2QWAmYPFQYKMjAxOC8wNS8wOAPtmZQCS1QG7IK87ISxBuyImOybkAUzLDY5OGQCtgEPZBYCZg8VBgoyMDE4LzA1LzA5A%2ByImAbrhKXshLwG7ZWc7ZmUBuqzoOyymQU3LDk4M2QCtwEPZBYCZg8VBgoyMDE4LzA1LzA5A%2ByImAJMRwbroa%2FrjbAG7J6g7IukBjE4LDU2MWQCuAEPZBYCZg8VBgoyMDE4LzA1LzA5A%2ByImANLSUEG65GQ7IKwBuq0keyjvAYxNCwzMTlkArkBD2QWAmYPFQYKMjAxOC8wNS8wOQPsiJgCTkMCU0sG66eI7IKwBTQsNDg0ZAK6AQ9kFgJmDxUGCjIwMTgvMDUvMDkD7IiYAktUBuyCvOyEsQbsiJjsm5AFNSw5ODVkArsBD2QWAmYPFQYKMjAxOC8wNS8xMAPrqqkG64Sl7IS8Bu2VnO2ZlAbqs6DsspkFOSw3MDNkArwBD2QWAmYPFQYKMjAxOC8wNS8xMAPrqqkCTEcG66Gv642wBuyeoOyLpAYxOSw2OTdkAr0BD2QWAmYPFQYKMjAxOC8wNS8xMAPrqqkDS0lBBuuRkOyCsAbqtJHso7wGMTMsMjQ1ZAK%2BAQ9kFgJmDxUGCjIwMTgvMDUvMTAD66qpAk5DAlNLBuuniOyCsAU3LDcxOGQCvwEPZBYCZg8VBgoyMDE4LzA1LzEwA%2BuqqQJLVAbsgrzshLEG7IiY7JuQBTUsOTc5ZALAAQ9kFgJmDxUGCjIwMTgvMDUvMTED6riIBuyCvOyEsQNLSUEG64yA6rWsBjEwLDM5MWQCwQEPZBYCZg8VBgoyMDE4LzA1LzExA%2Bq4iAbroa%2FrjbACS1QG7IKs7KeBBjE3LDAyNmQCwgEPZBYCZg8VBgoyMDE4LzA1LzExA%2Bq4iAJTSwJMRwbrrLjtlZkGMTUsODcwZALDAQ9kFgJmDxUGCjIwMTgvMDUvMTED6riIBu2VnO2ZlAJOQwbrjIDsoIQGMTIsMzEyZALEAQ9kFgJmDxUGCjIwMTgvMDUvMTED6riIBuuRkOyCsAbrhKXshLwG7J6g7IukBjE0LDI2OWQCxQEPZBYCZg8VBgoyMDE4LzA1LzEzA%2BydvAbsgrzshLEDS0lBBuuMgOq1rAYxNSwxMzVkAsYBD2QWAmYPFQYKMjAxOC8wNS8xMwPsnbwG66Gv642wAktUBuyCrOyngQYxOCw2NTBkAscBD2QWAmYPFQYKMjAxOC8wNS8xMwPsnbwCU0sCTEcG66y47ZWZBjI0LDQ5NmQCyAEPZBYCZg8VBgoyMDE4LzA1LzEzA%2BydvAbtlZztmZQCTkMG64yA7KCEBjEzLDAwMGQCyQEPZBYCZg8VBgoyMDE4LzA1LzEzA%2BydvAbrkZDsgrAG64Sl7IS8BuyeoOyLpAYxNyw4MTNkAsoBD2QWAmYPFQYKMjAxOC8wNS8xNQPtmZQG64Sl7IS8A0tJQQbqs6DsspkGMTIsMjUwZALLAQ9kFgJmDxUGCjIwMTgvMDUvMTUD7ZmUBu2VnO2ZlAJLVAbrjIDsoIQGMTAsNDU0ZALMAQ9kFgJmDxUGCjIwMTgvMDUvMTUD7ZmUBuyCvOyEsQJMRwbtj6ztla0FNiw3ODdkAs0BD2QWAmYPFQYKMjAxOC8wNS8xNQPtmZQCTkMG66Gv642wBuuniOyCsAYxMCwwMTJkAs4BD2QWAmYPFQYKMjAxOC8wNS8xNQPtmZQG65GQ7IKwAlNLBuyeoOyLpAYxMywyMjJkAs8BD2QWAmYPFQYKMjAxOC8wNS8xNgPsiJgG64Sl7IS8A0tJQQbqs6DsspkGMTEsMjczZALQAQ9kFgJmDxUGCjIwMTgvMDUvMTYD7IiYBu2VnO2ZlAJLVAbrjIDsoIQFNywwNTNkAtEBD2QWAmYPFQYKMjAxOC8wNS8xNgPsiJgG7IK87ISxAkxHBu2PrO2VrQU1LDcyMGQC0gEPZBYCZg8VBgoyMDE4LzA1LzE2A%2ByImAbrkZDsgrACU0sG7J6g7IukBTcsMTE5ZALTAQ9kFgJmDxUGCjIwMTgvMDUvMTcD66qpBuuEpeyEvANLSUEG6rOg7LKZBjExLDQ2NGQC1AEPZBYCZg8VBgoyMDE4LzA1LzE3A%2BuqqQbtlZztmZQCS1QG64yA7KCEBTYsNzcwZALVAQ9kFgJmDxUGCjIwMTgvMDUvMTcD66qpBuyCvOyEsQJMRwbtj6ztla0FNSw2MTRkAtYBD2QWAmYPFQYKMjAxOC8wNS8xNwPrqqkCTkMG66Gv642wBuuniOyCsAU3LDAzNWQC1wEPZBYCZg8VBgoyMDE4LzA1LzE4A%2Bq4iAJMRwbtlZztmZQG7J6g7IukBjIxLDQzMWQC2AEPZBYCZg8VBgoyMDE4LzA1LzE4A%2Bq4iAJLVAJOQwbsiJjsm5AFMyw3MzdkAtkBD2QWAmYPFQYKMjAxOC8wNS8xOAPquIgG66Gv642wBuuRkOyCsAbsgqzsp4EGMTEsMjg3ZALaAQ9kFgJmDxUGCjIwMTgvMDUvMTgD6riIA0tJQQJTSwbqtJHso7wGMTIsNjM3ZALbAQ9kFgJmDxUGCjIwMTgvMDUvMTgD6riIBuuEpeyEvAbsgrzshLEG6rOg7LKZBTMsMjk4ZALcAQ9kFgJmDxUGCjIwMTgvMDUvMTkD7YagAkxHBu2VnO2ZlAbsnqDsi6QGMjUsMDAwZALdAQ9kFgJmDxUGCjIwMTgvMDUvMTkD7YagAktUAk5DBuyImOybkAYxMSw3NTJkAt4BD2QWAmYPFQYKMjAxOC8wNS8xOQPthqAG66Gv642wBuuRkOyCsAbsgqzsp4EGMjUsMDAwZALfAQ9kFgJmDxUGCjIwMTgvMDUvMTkD7YagA0tJQQJTSwbqtJHso7wGMjAsNTAwZALgAQ9kFgJmDxUGCjIwMTgvMDUvMTkD7YagBuuEpeyEvAbsgrzshLEG6rOg7LKZBTgsNTQ3ZALhAQ9kFgJmDxUGCjIwMTgvMDUvMjAD7J28AkxHBu2VnO2ZlAbsnqDsi6QGMjUsMDAwZALiAQ9kFgJmDxUGCjIwMTgvMDUvMjAD7J28AktUAk5DBuyImOybkAU5LDQ5N2QC4wEPZBYCZg8VBgoyMDE4LzA1LzIwA%2BydvAbroa%2FrjbAG65GQ7IKwBuyCrOyngQYyNSwwMDBkAuQBD2QWAmYPFQYKMjAxOC8wNS8yMAPsnbwDS0lBAlNLBuq0keyjvAYyMCw1MDBkAuUBD2QWAmYPFQYKMjAxOC8wNS8yMAPsnbwG64Sl7IS8BuyCvOyEsQbqs6DsspkFNyw2MTBkAuYBD2QWAmYPFQYKMjAxOC8wNS8yMgPtmZQDS0lBAktUBuq0keyjvAYxOSw2OTNkAucBD2QWAmYPFQYKMjAxOC8wNS8yMgPtmZQG7IK87ISxBuuhr%2BuNsAbrjIDqtawGMjQsMDAwZALoAQ9kFgJmDxUGCjIwMTgvMDUvMjID7ZmUAkxHAk5DBuyeoOyLpAYxOCw1NDNkAukBD2QWAmYPFQYKMjAxOC8wNS8yMgPtmZQG7ZWc7ZmUBuuRkOyCsAbrjIDsoIQGMTMsMDAwZALqAQ9kFgJmDxUGCjIwMTgvMDUvMjID7ZmUAlNLBuuEpeyEvAbrrLjtlZkGMTksNjE2ZALrAQ9kFgJmDxUGCjIwMTgvMDUvMjMD7IiYA0tJQQJLVAbqtJHso7wGMTEsMzM0ZALsAQ9kFgJmDxUGCjIwMTgvMDUvMjMD7IiYBuyCvOyEsQbroa%2FrjbAG64yA6rWsBjEyLDAwOGQC7QEPZBYCZg8VBgoyMDE4LzA1LzIzA%2ByImAJMRwJOQwbsnqDsi6QFOSwyNzlkAu4BD2QWAmYPFQYKMjAxOC8wNS8yMwPsiJgG7ZWc7ZmUBuuRkOyCsAbrjIDsoIQGMTIsMzU3ZALvAQ9kFgJmDxUGCjIwMTgvMDUvMjMD7IiYAlNLBuuEpeyEvAbrrLjtlZkFNyw1ODJkAvABD2QWAmYPFQYKMjAxOC8wNS8yNAPrqqkDS0lBAktUBuq0keyjvAYxMSwyOTdkAvEBD2QWAmYPFQYKMjAxOC8wNS8yNAPrqqkG7IK87ISxBuuhr%2BuNsAbrjIDqtawGMTEsMjcwZALyAQ9kFgJmDxUGCjIwMTgvMDUvMjQD66qpAkxHAk5DBuyeoOyLpAYxMSwwNzVkAvMBD2QWAmYPFQYKMjAxOC8wNS8yNAPrqqkG7ZWc7ZmUBuuRkOyCsAbrjIDsoIQGMTMsMDAwZAL0AQ9kFgJmDxUGCjIwMTgvMDUvMjQD66qpAlNLBuuEpeyEvAbrrLjtlZkFNiw4NTZkAvUBD2QWAmYPFQYKMjAxOC8wNS8yNQPquIgCU0sG7ZWc7ZmUBuusuO2VmQYyMiw4ODlkAvYBD2QWAmYPFQYKMjAxOC8wNS8yNQPquIgCTkMDS0lBBuuniOyCsAU4LDA2NWQC9wEPZBYCZg8VBgoyMDE4LzA1LzI1A%2Bq4iAJLVAJMRwbsiJjsm5AGMTAsMDA5ZAL4AQ9kFgJmDxUGCjIwMTgvMDUvMjUD6riIBuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTYsMTMxZAL5AQ9kFgJmDxUGCjIwMTgvMDUvMjUD6riIBuuRkOyCsAbsgrzshLEG7J6g7IukBjE1LDE2OGQC%2BgEPZBYCZg8VBgoyMDE4LzA1LzI2A%2B2GoAJTSwbtlZztmZQG66y47ZWZBjI1LDAwMGQC%2BwEPZBYCZg8VBgoyMDE4LzA1LzI2A%2B2GoAJOQwNLSUEG66eI7IKwBjEwLDA4OGQC%2FAEPZBYCZg8VBgoyMDE4LzA1LzI2A%2B2GoAJLVAJMRwbsiJjsm5AGMTgsNDc5ZAL9AQ9kFgJmDxUGCjIwMTgvMDUvMjYD7YagBuuEpeyEvAbroa%2FrjbAG6rOg7LKZBjEyLDMzMWQC%2FgEPZBYCZg8VBgoyMDE4LzA1LzI2A%2B2GoAbrkZDsgrAG7IK87ISxBuyeoOyLpAYyNSwwMDBkAv8BD2QWAmYPFQYKMjAxOC8wNS8yNwPsnbwCU0sG7ZWc7ZmUBuusuO2VmQYyNSwwMDBkAoACD2QWAmYPFQYKMjAxOC8wNS8yNwPsnbwCTkMDS0lBBuuniOyCsAU4LDAxOWQCgQIPZBYCZg8VBgoyMDE4LzA1LzI3A%2BydvAJLVAJMRwbsiJjsm5AGMTYsMjMyZAKCAg9kFgJmDxUGCjIwMTgvMDUvMjcD7J28BuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTksOTY0ZAKDAg9kFgJmDxUGCjIwMTgvMDUvMjcD7J28BuuRkOyCsAbsgrzshLEG7J6g7IukBjIxLDE3MWQChAIPZBYCZg8VBgoyMDE4LzA1LzI5A%2B2ZlAbsgrzshLECS1QG64yA6rWsBTcsMzQyZAKFAg9kFgJmDxUGCjIwMTgvMDUvMjkD7ZmUBuuhr%2BuNsAJMRwbsgqzsp4EGMTQsNzc1ZAKGAg9kFgJmDxUGCjIwMTgvMDUvMjkD7ZmUBu2VnO2ZlAJOQwbrjIDsoIQFOSwxNThkAocCD2QWAmYPFQYKMjAxOC8wNS8yOQPtmZQDS0lBBuuEpeyEvAbqtJHso7wFOSwxOTlkAogCD2QWAmYPFQYKMjAxOC8wNS8zMAPsiJgG7IK87ISxAktUBuuMgOq1rAU3LDcwNWQCiQIPZBYCZg8VBgoyMDE4LzA1LzMwA%2ByImAbroa%2FrjbACTEcG7IKs7KeBBjE1LDMwOWQCigIPZBYCZg8VBgoyMDE4LzA1LzMwA%2ByImAbtlZztmZQCTkMG64yA7KCEBjEwLDQ1OGQCiwIPZBYCZg8VBgoyMDE4LzA1LzMwA%2ByImAbrkZDsgrACU0sG7J6g7IukBjEyLDM4MmQCjAIPZBYCZg8VBgoyMDE4LzA1LzMwA%2ByImANLSUEG64Sl7IS8Buq0keyjvAU5LDU5MWQCjQIPZBYCZg8VBgoyMDE4LzA1LzMxA%2BuqqQbsgrzshLECS1QG64yA6rWsBTgsMjE4ZAKOAg9kFgJmDxUGCjIwMTgvMDUvMzED66qpBuuhr%2BuNsAJMRwbsgqzsp4EGMTAsMzg5ZAKPAg9kFgJmDxUGCjIwMTgvMDUvMzED66qpBu2VnO2ZlAJOQwbrjIDsoIQGMTEsNDc5ZAKQAg9kFgJmDxUGCjIwMTgvMDUvMzED66qpBuuRkOyCsAJTSwbsnqDsi6QGMTIsNDEyZAKRAg9kFgJmDxUGCjIwMTgvMDUvMzED66qpA0tJQQbrhKXshLwG6rSR7KO8BTksOTI4ZAKSAg9kFgJmDxUGCjIwMTgvMDYvMDED6riIBuuhr%2BuNsAbtlZztmZQG7IKs7KeBBjEzLDY0MWQCkwIPZBYCZg8VBgoyMDE4LzA2LzAxA%2Bq4iAJTSwJLVAbrrLjtlZkGMTQsNDUxZAKUAg9kFgJmDxUGCjIwMTgvMDYvMDED6riIA0tJQQbrkZDsgrAG6rSR7KO8BjE3LDExNGQClQIPZBYCZg8VBgoyMDE4LzA2LzAxA%2Bq4iAJOQwbsgrzshLEG66eI7IKwBTUsNzYzZAKWAg9kFgJmDxUGCjIwMTgvMDYvMDED6riIAkxHBuuEpeyEvAbsnqDsi6QGMTIsMzg1ZAKXAg9kFgJmDxUGCjIwMTgvMDYvMDID7YagBuuhr%2BuNsAbtlZztmZQG7IKs7KeBBjI1LDAwMGQCmAIPZBYCZg8VBgoyMDE4LzA2LzAyA%2B2GoAJTSwJLVAbrrLjtlZkGMTgsNDY1ZAKZAg9kFgJmDxUGCjIwMTgvMDYvMDID7YagA0tJQQbrkZDsgrAG6rSR7KO8BjIwLDUwMGQCmgIPZBYCZg8VBgoyMDE4LzA2LzAyA%2B2GoAJOQwbsgrzshLEG66eI7IKwBTgsMDM3ZAKbAg9kFgJmDxUGCjIwMTgvMDYvMDID7YagAkxHBuuEpeyEvAbsnqDsi6QGMjAsMjI3ZAKcAg9kFgJmDxUGCjIwMTgvMDYvMDMD7J28Buuhr%2BuNsAbtlZztmZQG7IKs7KeBBjE0LDQwOGQCnQIPZBYCZg8VBgoyMDE4LzA2LzAzA%2BydvAJTSwJLVAbrrLjtlZkGMTQsMDI4ZAKeAg9kFgJmDxUGCjIwMTgvMDYvMDMD7J28A0tJQQbrkZDsgrAG6rSR7KO8BjE4LDU4MWQCnwIPZBYCZg8VBgoyMDE4LzA2LzAzA%2BydvAJOQwbsgrzshLEG66eI7IKwBTYsMTU1ZAKgAg9kFgJmDxUGCjIwMTgvMDYvMDMD7J28AkxHBuuEpeyEvAbsnqDsi6QGMTUsNjg3ZAKhAg9kFgJmDxUGCjIwMTgvMDYvMDUD7ZmUAkxHBu2VnO2ZlAbsnqDsi6QGMjMsOTM0ZAKiAg9kFgJmDxUGCjIwMTgvMDYvMDUD7ZmUAktUA0tJQQbsiJjsm5AGMTIsNTk4ZAKjAg9kFgJmDxUGCjIwMTgvMDYvMDUD7ZmUAk5DBuuhr%2BuNsAbrp4jsgrAFOCw1MzZkAqQCD2QWAmYPFQYKMjAxOC8wNi8wNQPtmZQG64Sl7IS8BuuRkOyCsAbqs6DsspkFNiw2NTVkAqUCD2QWAmYPFQYKMjAxOC8wNi8wNQPtmZQCU0sG7IK87ISxBuusuO2VmQYxMCw4MjhkAqYCD2QWAmYPFQYKMjAxOC8wNi8wNgPsiJgCTEcG7ZWc7ZmUBuyeoOyLpAYyMyw1OTNkAqcCD2QWAmYPFQYKMjAxOC8wNi8wNgPsiJgCS1QDS0lBBuyImOybkAYyMCw4MDBkAqgCD2QWAmYPFQYKMjAxOC8wNi8wNgPsiJgCTkMG66Gv642wBuuniOyCsAU4LDg2N2QCqQIPZBYCZg8VBgoyMDE4LzA2LzA2A%2ByImAbrhKXshLwG65GQ7IKwBuqzoOyymQYxMyw3MzFkAqoCD2QWAmYPFQYKMjAxOC8wNi8wNgPsiJgCU0sG7IK87ISxBuusuO2VmQYyMCwzODFkAqsCD2QWAmYPFQYKMjAxOC8wNi8wNwPrqqkCTEcG7ZWc7ZmUBuyeoOyLpAYyMCwyNDVkAqwCD2QWAmYPFQYKMjAxOC8wNi8wNwPrqqkCS1QDS0lBBuyImOybkAYxMCw0MzZkAq0CD2QWAmYPFQYKMjAxOC8wNi8wNwPrqqkCTkMG66Gv642wBuuniOyCsAU3LDU3M2QCrgIPZBYCZg8VBgoyMDE4LzA2LzA3A%2BuqqQbrhKXshLwG65GQ7IKwBuqzoOyymQU0LDEwOGQCrwIPZBYCZg8VBgoyMDE4LzA2LzA3A%2BuqqQJTSwbsgrzshLEG66y47ZWZBTcsNjI0ZAKwAg9kFgJmDxUGCjIwMTgvMDYvMDgD6riIBuuhr%2BuNsANLSUEG7IKs7KeBBjEzLDI0NWQCsQIPZBYCZg8VBgoyMDE4LzA2LzA4A%2Bq4iAbsgrzshLECTEcG64yA6rWsBjE0LDYxM2QCsgIPZBYCZg8VBgoyMDE4LzA2LzA4A%2Bq4iAbrkZDsgrACTkMG7J6g7IukBjExLDUzMWQCswIPZBYCZg8VBgoyMDE4LzA2LzA4A%2Bq4iAbtlZztmZQCU0sG64yA7KCEBjEzLDAwMGQCtAIPZBYCZg8VBgoyMDE4LzA2LzA4A%2Bq4iAJLVAbrhKXshLwG7IiY7JuQBTMsNDkyZAK1Ag9kFgJmDxUGCjIwMTgvMDYvMDkD7YagBuuhr%2BuNsANLSUEG7IKs7KeBBjI1LDAwMGQCtgIPZBYCZg8VBgoyMDE4LzA2LzA5A%2B2GoAbsgrzshLECTEcG64yA6rWsBjI0LDAwMGQCtwIPZBYCZg8VBgoyMDE4LzA2LzA5A%2B2GoAbrkZDsgrACTkMG7J6g7IukBjIwLDMxNGQCuAIPZBYCZg8VBgoyMDE4LzA2LzA5A%2B2GoAbtlZztmZQCU0sG64yA7KCEBjEzLDAwMGQCuQIPZBYCZg8VBgoyMDE4LzA2LzA5A%2B2GoAJLVAbrhKXshLwG7IiY7JuQBjEwLDIzMGQCugIPZBYCZg8VBgoyMDE4LzA2LzEwA%2BydvAbsgrzshLECTEcG64yA6rWsBjEzLDU5NGQCuwIPZBYCZg8VBgoyMDE4LzA2LzEwA%2BydvAbrkZDsgrACTkMG7J6g7IukBjE0LDMyM2QCvAIPZBYCZg8VBgoyMDE4LzA2LzEwA%2BydvAbtlZztmZQCU0sG64yA7KCEBjEzLDAwMGQCvQIPZBYCZg8VBgoyMDE4LzA2LzEwA%2BydvAJLVAbrhKXshLwG7IiY7JuQBTcsMTg2ZAK%2BAg9kFgJmDxUGCjIwMTgvMDYvMTID7ZmUBuuEpeyEvAbtlZztmZQG6rOg7LKZBTksNTI3ZAK%2FAg9kFgJmDxUGCjIwMTgvMDYvMTID7ZmUBuuRkOyCsAJLVAbsnqDsi6QGMTAsNDEyZALAAg9kFgJmDxUGCjIwMTgvMDYvMTID7ZmUAk5DAkxHBuuniOyCsAU1LDczMGQCwQIPZBYCZg8VBgoyMDE4LzA2LzEyA%2B2ZlANLSUECU0sG6rSR7KO8BjE2LDUxMmQCwgIPZBYCZg8VBgoyMDE4LzA2LzEyA%2B2ZlAbroa%2FrjbAG7IK87ISxBuyCrOyngQYxNSw1NDFkAsMCD2QWAmYPFQYKMjAxOC8wNi8xMwPsiJgG64Sl7IS8Bu2VnO2ZlAbqs6DsspkGMTIsNzU2ZALEAg9kFgJmDxUGCjIwMTgvMDYvMTMD7IiYBuuRkOyCsAJLVAbsnqDsi6QGMTMsODQ1ZALFAg9kFgJmDxUGCjIwMTgvMDYvMTMD7IiYAk5DAkxHBuuniOyCsAU0LDgzOGQCxgIPZBYCZg8VBgoyMDE4LzA2LzEzA%2ByImANLSUECU0sG6rSR7KO8BjE2LDg2OWQCxwIPZBYCZg8VBgoyMDE4LzA2LzEzA%2ByImAbroa%2FrjbAG7IK87ISxBuyCrOyngQYxMyw1OTJkAsgCD2QWAmYPFQYKMjAxOC8wNi8xNAPrqqkG64Sl7IS8Bu2VnO2ZlAbqs6DsspkFNyw2NTZkAskCD2QWAmYPFQYKMjAxOC8wNi8xNAPrqqkG65GQ7IKwAktUBuyeoOyLpAU3LDUxM2QCygIPZBYCZg8VBgoyMDE4LzA2LzE0A%2BuqqQJOQwJMRwbrp4jsgrAFNCw0NjNkAssCD2QWAmYPFQYKMjAxOC8wNi8xNAPrqqkDS0lBAlNLBuq0keyjvAYxMCwzMTdkAswCD2QWAmYPFQYKMjAxOC8wNi8xNAPrqqkG66Gv642wBuyCvOyEsQbsgqzsp4EFOSwzMThkAs0CD2QWAmYPFQYKMjAxOC8wNi8xNQPquIgCTEcDS0lBBuyeoOyLpAYyMywzNzVkAs4CD2QWAmYPFQYKMjAxOC8wNi8xNQPquIgCTkMCS1QG66eI7IKwBTQsNDE4ZALPAg9kFgJmDxUGCjIwMTgvMDYvMTUD6riIAlNLBuuhr%2BuNsAbrrLjtlZkGMTMsMjA0ZALQAg9kFgJmDxUGCjIwMTgvMDYvMTUD6riIBu2VnO2ZlAbrkZDsgrAG64yA7KCEBjEzLDAwMGQC0QIPZBYCZg8VBgoyMDE4LzA2LzE1A%2Bq4iAbrhKXshLwG7IK87ISxBuqzoOyymQUyLDY3NmQC0gIPZBYCZg8VBgoyMDE4LzA2LzE2A%2B2GoAJMRwNLSUEG7J6g7IukBjI1LDAwMGQC0wIPZBYCZg8VBgoyMDE4LzA2LzE2A%2B2GoAJOQwJLVAbrp4jsgrAFNyw1MDNkAtQCD2QWAmYPFQYKMjAxOC8wNi8xNgPthqACU0sG66Gv642wBuusuO2VmQYyMywwNDdkAtUCD2QWAmYPFQYKMjAxOC8wNi8xNgPthqAG7ZWc7ZmUBuuRkOyCsAbrjIDsoIQGMTMsMDAwZALWAg9kFgJmDxUGCjIwMTgvMDYvMTYD7YagBuuEpeyEvAbsgrzshLEG6rOg7LKZBTcsODI3ZALXAg9kFgJmDxUGCjIwMTgvMDYvMTcD7J28AkxHA0tJQQbsnqDsi6QGMjMsNTg2ZALYAg9kFgJmDxUGCjIwMTgvMDYvMTcD7J28Ak5DAktUBuuniOyCsAU2LDU5NmQC2QIPZBYCZg8VBgoyMDE4LzA2LzE3A%2BydvAJTSwbroa%2FrjbAG66y47ZWZBjE2LDQwM2QC2gIPZBYCZg8VBgoyMDE4LzA2LzE3A%2BydvAbtlZztmZQG65GQ7IKwBuuMgOyghAYxMywwMDBkAtsCD2QWAmYPFQYKMjAxOC8wNi8xNwPsnbwG64Sl7IS8BuyCvOyEsQbqs6DsspkFNCwwODlkAtwCD2QWAmYPFQYKMjAxOC8wNi8xOQPtmZQG7ZWc7ZmUAkxHBuyyreyjvAYxMCwwMDBkAt0CD2QWAmYPFQYKMjAxOC8wNi8xOQPtmZQCS1QG66Gv642wBuyImOybkAU2LDI5OWQC3gIPZBYCZg8VBgoyMDE4LzA2LzE5A%2B2ZlAbsgrzshLECU0sG64yA6rWsBTQsMzUzZALfAg9kFgJmDxUGCjIwMTgvMDYvMTkD7ZmUBuuRkOyCsAbrhKXshLwG7J6g7IukBTcsMzIzZALgAg9kFgJmDxUGCjIwMTgvMDYvMjAD7IiYBu2VnO2ZlAJMRwbssq3so7wGMTAsMDAwZALhAg9kFgJmDxUGCjIwMTgvMDYvMjAD7IiYAktUBuuhr%2BuNsAbsiJjsm5AFNiw0MDRkAuICD2QWAmYPFQYKMjAxOC8wNi8yMAPsiJgDS0lBAk5DBuq0keyjvAU3LDkxNmQC4wIPZBYCZg8VBgoyMDE4LzA2LzIwA%2ByImAbsgrzshLECU0sG64yA6rWsBTYsNTk0ZALkAg9kFgJmDxUGCjIwMTgvMDYvMjAD7IiYBuuRkOyCsAbrhKXshLwG7J6g7IukBTksMDU2ZALlAg9kFgJmDxUGCjIwMTgvMDYvMjED66qpBu2VnO2ZlAJMRwbssq3so7wGMTAsMDAwZALmAg9kFgJmDxUGCjIwMTgvMDYvMjED66qpAktUBuuhr%2BuNsAbsiJjsm5AFNywwMjNkAucCD2QWAmYPFQYKMjAxOC8wNi8yMQPrqqkDS0lBAk5DBuq0keyjvAYxNyw4OTJkAugCD2QWAmYPFQYKMjAxOC8wNi8yMQPrqqkG7IK87ISxAlNLBuuMgOq1rAU2LDI0M2QC6QIPZBYCZg8VBgoyMDE4LzA2LzIxA%2BuqqQbrkZDsgrAG64Sl7IS8BuyeoOyLpAYxMCwxNDVkAuoCD2QWAmYPFQYKMjAxOC8wNi8yMgPquIgCTkMG7ZWc7ZmUBuuniOyCsAU3LDUyNWQC6wIPZBYCZg8VBgoyMDE4LzA2LzIyA%2Bq4iAbrhKXshLwDS0lBBuqzoOyymQU2LDc4OGQC7AIPZBYCZg8VBgoyMDE4LzA2LzIyA%2Bq4iAJMRwbroa%2FrjbAG7J6g7IukBjIxLDA4NGQC7QIPZBYCZg8VBgoyMDE4LzA2LzIyA%2Bq4iAbsgrzshLEG65GQ7IKwBuuMgOq1rAYxMiw0MDVkAu4CD2QWAmYPFQYKMjAxOC8wNi8yMgPquIgCS1QCU0sG7IiY7JuQBTMsNzk3ZALvAg9kFgJmDxUGCjIwMTgvMDYvMjMD7YagAk5DBu2VnO2ZlAbrp4jsgrAFOSw1NDlkAvACD2QWAmYPFQYKMjAxOC8wNi8yMwPthqAG64Sl7IS8A0tJQQbqs6DsspkGMTQsODExZALxAg9kFgJmDxUGCjIwMTgvMDYvMjMD7YagAkxHBuuhr%2BuNsAbsnqDsi6QGMjQsMjEyZALyAg9kFgJmDxUGCjIwMTgvMDYvMjMD7YagBuyCvOyEsQbrkZDsgrAG64yA6rWsBjE1LDMzMmQC8wIPZBYCZg8VBgoyMDE4LzA2LzIzA%2B2GoAJLVAJTSwbsiJjsm5AFOSw2NzNkAvQCD2QWAmYPFQYKMjAxOC8wNi8yNAPsnbwCTkMG7ZWc7ZmUBuuniOyCsAU2LDEzNmQC9QIPZBYCZg8VBgoyMDE4LzA2LzI0A%2BydvAbrhKXshLwDS0lBBuqzoOyymQYxMCwyMjdkAvYCD2QWAmYPFQYKMjAxOC8wNi8yNAPsnbwCTEcG66Gv642wBuyeoOyLpAYyMCw5MDRkAvcCD2QWAmYPFQYKMjAxOC8wNi8yNAPsnbwG7IK87ISxBuuRkOyCsAbrjIDqtawGMTAsMTg4ZAL4Ag9kFgJmDxUGCjIwMTgvMDYvMjQD7J28AktUAlNLBuyImOybkAU2LDQ2OGQC%2BQIPZBYCZg8VBgoyMDE4LzA2LzI2A%2B2ZlAJOQwbrkZDsgrAG66eI7IKwBTIsOTIyZAL6Ag9kFgJmDxUGCjIwMTgvMDYvMjYD7ZmUBu2VnO2ZlAbsgrzshLEG64yA7KCEBTcsMzQ0ZAL7Ag9kFgJmDxUGCjIwMTgvMDYvMjYD7ZmUBuuhr%2BuNsAbrhKXshLwG7IKs7KeBBTksNTA3ZAL8Ag9kFgJmDxUGCjIwMTgvMDYvMjcD7IiYAlNLA0tJQQbrrLjtlZkGMTIsNDA0ZAL9Ag9kFgJmDxUGCjIwMTgvMDYvMjcD7IiYAkxHAktUBuyeoOyLpAU5LDAwNGQC%2FgIPZBYCZg8VBgoyMDE4LzA2LzI3A%2ByImAJOQwbrkZDsgrAG66eI7IKwBTIsODY1ZAL%2FAg9kFgJmDxUGCjIwMTgvMDYvMjcD7IiYBu2VnO2ZlAbsgrzshLEG64yA7KCEBTgsMTUzZAKAAw9kFgJmDxUGCjIwMTgvMDYvMjcD7IiYBuuhr%2BuNsAbrhKXshLwG7IKs7KeBBTQsMjY0ZAKBAw9kFgJmDxUGCjIwMTgvMDYvMjgD66qpAk5DBuuRkOyCsAbrp4jsgrAFMiw3NjhkAoIDD2QWAmYPFQYKMjAxOC8wNi8yOAPrqqkG7ZWc7ZmUBuyCvOyEsQbrjIDsoIQFOSw5MTRkAoMDD2QWAmYPFQYKMjAxOC8wNi8yOAPrqqkG66Gv642wBuuEpeyEvAbsgqzsp4EFNCw1MTBkAoQDD2QWAmYPFQYKMjAxOC8wNi8yOQPquIgG65GQ7IKwA0tJQQbsnqDsi6QGMjMsMTE0ZAKFAw9kFgJmDxUGCjIwMTgvMDYvMjkD6riIAlNLAkxHBuusuO2VmQYxNCwwNTRkAoYDD2QWAmYPFQYKMjAxOC8wNi8yOQPquIgG7ZWc7ZmUBuuhr%2BuNsAbrjIDsoIQGMTMsMDAwZAKHAw9kFgJmDxUGCjIwMTgvMDYvMjkD6riIAktUAk5DBuyImOybkAUzLDI2NmQCiAMPZBYCZg8VBgoyMDE4LzA2LzI5A%2Bq4iAbsgrzshLEG64Sl7IS8BuuMgOq1rAU4LDAyMGQCiQMPZBYCZg8VBgoyMDE4LzA2LzMwA%2B2GoAbrkZDsgrADS0lBBuyeoOyLpAYyNCwxMTBkAooDD2QWAmYPFQYKMjAxOC8wNi8zMAPthqACU0sCTEcG66y47ZWZBjE5LDAyNGQCiwMPZBYCZg8VBgoyMDE4LzA2LzMwA%2B2GoAbtlZztmZQG66Gv642wBuuMgOyghAYxMSw4OTFkAowDD2QWAmYPFQYKMjAxOC8wNi8zMAPthqACS1QCTkMG7IiY7JuQBTQsMjM3ZAKNAw9kFgJmDxUGCjIwMTgvMDcvMDED7J28BuyCvOyEsQbrhKXshLwG64yA6rWsBTUsMDE1ZAKOAw9kFgJmDxUGCjIwMTgvMDcvMDMD7ZmUA0tJQQbtlZztmZQG6rSR7KO8BTUsNDg0ZAKPAw9kFgJmDxUGCjIwMTgvMDcvMDMD7ZmUAkxHAk5DBuyeoOyLpAU3LDAzNGQCkAMPZBYCZg8VBgoyMDE4LzA3LzAzA%2B2ZlAbrhKXshLwCU0sG6rOg7LKZBTIsNjM1ZAKRAw9kFgJmDxUGCjIwMTgvMDcvMDQD7IiYA0tJQQbtlZztmZQG6rSR7KO8BTcsMjg3ZAKSAw9kFgJmDxUGCjIwMTgvMDcvMDQD7IiYAkxHAk5DBuyeoOyLpAU3LDM0M2QCkwMPZBYCZg8VBgoyMDE4LzA3LzA0A%2ByImAbroa%2FrjbAG65GQ7IKwBuyCrOyngQU3LDc1MGQClAMPZBYCZg8VBgoyMDE4LzA3LzA0A%2ByImAbrhKXshLwCU0sG6rOg7LKZBTIsMzA1ZAKVAw9kFgJmDxUGCjIwMTgvMDcvMDQD7IiYAktUBuyCvOyEsQbsiJjsm5AFMywzMjBkApYDD2QWAmYPFQYKMjAxOC8wNy8wNQPrqqkDS0lBBu2VnO2ZlAbqtJHso7wFNiw5MjNkApcDD2QWAmYPFQYKMjAxOC8wNy8wNQPrqqkCTEcCTkMG7J6g7IukBTgsMTI1ZAKYAw9kFgJmDxUGCjIwMTgvMDcvMDUD66qpBuuhr%2BuNsAbrkZDsgrAG7IKs7KeBBTcsODk2ZAKZAw9kFgJmDxUGCjIwMTgvMDcvMDUD66qpBuuEpeyEvAJTSwbqs6DsspkFMiw4NzZkApoDD2QWAmYPFQYKMjAxOC8wNy8wNQPrqqkCS1QG7IK87ISxBuyImOybkAUzLDE4OWQCmwMPZBYCZg8VBgoyMDE4LzA3LzA2A%2Bq4iAJTSwbtlZztmZQG66y47ZWZBjE4LDg4MmQCnAMPZBYCZg8VBgoyMDE4LzA3LzA2A%2Bq4iAbroa%2FrjbACS1QG7IKs7KeBBjEwLDQ5N2QCnQMPZBYCZg8VBgoyMDE4LzA3LzA2A%2Bq4iANLSUECTEcG6rSR7KO8BjEwLDEzNmQCngMPZBYCZg8VBgoyMDE4LzA3LzA2A%2Bq4iAbrhKXshLwCTkMG6rOg7LKZBTEsNzU1ZAKfAw9kFgJmDxUGCjIwMTgvMDcvMDYD6riIBuuRkOyCsAbsgrzshLEG7J6g7IukBjE3LDI2MmQCoAMPZBYCZg8VBgoyMDE4LzA3LzA3A%2B2GoAJTSwbtlZztmZQG66y47ZWZBjI1LDAwMGQCoQMPZBYCZg8VBgoyMDE4LzA3LzA3A%2B2GoAbroa%2FrjbACS1QG7IKs7KeBBjIyLDk1OGQCogMPZBYCZg8VBgoyMDE4LzA3LzA3A%2B2GoANLSUECTEcG6rSR7KO8BjE5LDM1MmQCowMPZBYCZg8VBgoyMDE4LzA3LzA3A%2B2GoAbrhKXshLwCTkMG6rOg7LKZBTQsNjgxZAKkAw9kFgJmDxUGCjIwMTgvMDcvMDcD7YagBuuRkOyCsAbsgrzshLEG7J6g7IukBjIxLDEzOGQCpQMPZBYCZg8VBgoyMDE4LzA3LzA4A%2BydvAJTSwbtlZztmZQG66y47ZWZBjIzLDA0NGQCpgMPZBYCZg8VBgoyMDE4LzA3LzA4A%2BydvAbroa%2FrjbACS1QG7IKs7KeBBjExLDUyOGQCpwMPZBYCZg8VBgoyMDE4LzA3LzA4A%2BydvANLSUECTEcG6rSR7KO8BjE1LDAzOGQCqAMPZBYCZg8VBgoyMDE4LzA3LzA4A%2BydvAbrhKXshLwCTkMG6rOg7LKZBTIsMzEyZAKpAw9kFgJmDxUGCjIwMTgvMDcvMDgD7J28BuuRkOyCsAbsgrzshLEG7J6g7IukBjE3LDE0NGQCqgMPZBYCZg8VBgoyMDE4LzA3LzEwA%2B2ZlAJOQwNLSUEG66eI7IKwBTUsMDg5ZAKrAw9kFgJmDxUGCjIwMTgvMDcvMTAD7ZmUBuyCvOyEsQbroa%2FrjbAG7Y%2Bs7ZWtBTgsMjQ1ZAKsAw9kFgJmDxUGCjIwMTgvMDcvMTAD7ZmUAktUBuuRkOyCsAbsiJjsm5AFNSwwMzBkAq0DD2QWAmYPFQYKMjAxOC8wNy8xMAPtmZQCTEcCU0sG7J6g7IukBTgsNDQ1ZAKuAw9kFgJmDxUGCjIwMTgvMDcvMTAD7ZmUBu2VnO2ZlAbrhKXshLwG64yA7KCEBTksNzc2ZAKvAw9kFgJmDxUGCjIwMTgvMDcvMTED7IiYAk5DA0tJQQbrp4jsgrAFNSw2MzFkArADD2QWAmYPFQYKMjAxOC8wNy8xMQPsiJgG7IK87ISxBuuhr%2BuNsAbtj6ztla0FOCw5MjBkArEDD2QWAmYPFQYKMjAxOC8wNy8xMQPsiJgCS1QG65GQ7IKwBuyImOybkAU2LDI0OWQCsgMPZBYCZg8VBgoyMDE4LzA3LzExA%2ByImAJMRwJTSwbsnqDsi6QFOSw0MTdkArMDD2QWAmYPFQYKMjAxOC8wNy8xMQPsiJgG7ZWc7ZmUBuuEpeyEvAbrjIDsoIQGMTAsMDA3ZAK0Aw9kFgJmDxUGCjIwMTgvMDcvMTID66qpAk5DA0tJQQbrp4jsgrAFNCw3NjFkArUDD2QWAmYPFQYKMjAxOC8wNy8xMgPrqqkG7IK87ISxBuuhr%2BuNsAbtj6ztla0FOCw0NzJkArYDD2QWAmYPFQYKMjAxOC8wNy8xMgPrqqkCS1QG65GQ7IKwBuyImOybkAU2LDMwNmQCtwMPZBYCZg8VBgoyMDE4LzA3LzEyA%2BuqqQJMRwJTSwbsnqDsi6QFOSw5NDdkArgDD2QWAmYPFQYKMjAxOC8wNy8xMgPrqqkG7ZWc7ZmUBuuEpeyEvAbrjIDsoIQGMTAsNzQxZAK5Aw9kFgJmDxUGCjIwMTgvMDcvMTcD7ZmUAktUBu2VnO2ZlAbsiJjsm5AFOSw3MjlkAroDD2QWAmYPFQYKMjAxOC8wNy8xNwPtmZQG64Sl7IS8AkxHBuqzoOyymQU1LDY4OGQCuwMPZBYCZg8VBgoyMDE4LzA3LzE3A%2B2ZlAbrkZDsgrAG66Gv642wBuyeoOyLpAYxMiwxMjJkArwDD2QWAmYPFQYKMjAxOC8wNy8xNwPtmZQCU0sCTkMG66y47ZWZBTUsMTk1ZAK9Aw9kFgJmDxUGCjIwMTgvMDcvMTcD7ZmUA0tJQQbsgrzshLEG6rSR7KO8BTYsNjg1ZAK%2BAw9kFgJmDxUGCjIwMTgvMDcvMTgD7IiYAktUBu2VnO2ZlAbsiJjsm5AFOSw1OTlkAr8DD2QWAmYPFQYKMjAxOC8wNy8xOAPsiJgG64Sl7IS8AkxHBuqzoOyymQU1LDY3MGQCwAMPZBYCZg8VBgoyMDE4LzA3LzE4A%2ByImAbrkZDsgrAG66Gv642wBuyeoOyLpAYxMSw3MDdkAsEDD2QWAmYPFQYKMjAxOC8wNy8xOAPsiJgCU0sCTkMG66y47ZWZBTYsNTYzZALCAw9kFgJmDxUGCjIwMTgvMDcvMTgD7IiYA0tJQQbsgrzshLEG6rSR7KO8BTcsMDU4ZALDAw9kFgJmDxUGCjIwMTgvMDcvMTkD66qpAktUBu2VnO2ZlAbsiJjsm5AFOSw5MThkAsQDD2QWAmYPFQYKMjAxOC8wNy8xOQPrqqkG64Sl7IS8AkxHBuqzoOyymQU2LDQ3NWQCxQMPZBYCZg8VBgoyMDE4LzA3LzE5A%2BuqqQbrkZDsgrAG66Gv642wBuyeoOyLpAYxMiwyNjVkAsYDD2QWAmYPFQYKMjAxOC8wNy8xOQPrqqkCU0sCTkMG66y47ZWZBTYsNzAyZALHAw9kFgJmDxUGCjIwMTgvMDcvMTkD66qpA0tJQQbsgrzshLEG6rSR7KO8BTgsODM0ZALIAw9kFgJmDxUGCjIwMTgvMDcvMjAD6riIBuyCvOyEsQbtlZztmZQG64yA6rWsBjEwLDEwMWQCyQMPZBYCZg8VBgoyMDE4LzA3LzIwA%2Bq4iANLSUECS1QG6rSR7KO8BTksNDg3ZALKAw9kFgJmDxUGCjIwMTgvMDcvMjAD6riIAkxHBuuRkOyCsAbsnqDsi6QGMTksODAwZALLAw9kFgJmDxUGCjIwMTgvMDcvMjAD6riIBuuhr%2BuNsAJTSwbsgqzsp4EFOSwyOTNkAswDD2QWAmYPFQYKMjAxOC8wNy8yMAPquIgCTkMG64Sl7IS8BuuniOyCsAU0LDU3N2QCzQMPZBYCZg8VBgoyMDE4LzA3LzIxA%2B2GoAbsgrzshLEG7ZWc7ZmUBuuMgOq1rAYxMiwxMDFkAs4DD2QWAmYPFQYKMjAxOC8wNy8yMQPthqADS0lBAktUBuq0keyjvAYxMywxNzdkAs8DD2QWAmYPFQYKMjAxOC8wNy8yMQPthqACTEcG65GQ7IKwBuyeoOyLpAYyMiw1NTBkAtADD2QWAmYPFQYKMjAxOC8wNy8yMQPthqAG66Gv642wAlNLBuyCrOyngQYxNSwzNzFkAtEDD2QWAmYPFQYKMjAxOC8wNy8yMQPthqACTkMG64Sl7IS8BuuniOyCsAU3LDA0OGQC0gMPZBYCZg8VBgoyMDE4LzA3LzIyA%2BydvAbsgrzshLEG7ZWc7ZmUBuuMgOq1rAU4LDE0NmQC0wMPZBYCZg8VBgoyMDE4LzA3LzIyA%2BydvANLSUECS1QG6rSR7KO8BTgsMDE4ZALUAw9kFgJmDxUGCjIwMTgvMDcvMjID7J28AkxHBuuRkOyCsAbsnqDsi6QGMTgsNDYzZALVAw9kFgJmDxUGCjIwMTgvMDcvMjID7J28Buuhr%2BuNsAJTSwbsgqzsp4EFNyw2MDlkAtYDD2QWAmYPFQYKMjAxOC8wNy8yMgPsnbwCTkMG64Sl7IS8BuuniOyCsAU0LDM4MWQC1wMPZBYCZg8VBgoyMDE4LzA3LzI0A%2B2ZlAbtlZztmZQDS0lBBuuMgOyghAU5LDYwNGQC2AMPZBYCZg8VBgoyMDE4LzA3LzI0A%2B2ZlAbrhKXshLwCS1QG6rOg7LKZBTEsNTE1ZALZAw9kFgJmDxUGCjIwMTgvMDcvMjQD7ZmUBuuhr%2BuNsAJOQwbsgqzsp4EFNyw1NTlkAtoDD2QWAmYPFQYKMjAxOC8wNy8yNAPtmZQCU0sG65GQ7IKwBuusuO2VmQYxNSw0MzNkAtsDD2QWAmYPFQYKMjAxOC8wNy8yNAPtmZQCTEcG7IK87ISxBuyeoOyLpAU5LDM5M2QC3AMPZBYCZg8VBgoyMDE4LzA3LzI1A%2ByImAbtlZztmZQDS0lBBuuMgOyghAU5LDI4MWQC3QMPZBYCZg8VBgoyMDE4LzA3LzI1A%2ByImAbrhKXshLwCS1QG6rOg7LKZBTEsNjAwZALeAw9kFgJmDxUGCjIwMTgvMDcvMjUD7IiYBuuhr%2BuNsAJOQwbsgqzsp4EFNywzMThkAt8DD2QWAmYPFQYKMjAxOC8wNy8yNQPsiJgCU0sG65GQ7IKwBuusuO2VmQYxMywwNjhkAuADD2QWAmYPFQYKMjAxOC8wNy8yNQPsiJgCTEcG7IK87ISxBuyeoOyLpAU4LDg0OWQC4QMPZBYCZg8VBgoyMDE4LzA3LzI2A%2BuqqQbtlZztmZQDS0lBBuuMgOyghAU5LDM5N2QC4gMPZBYCZg8VBgoyMDE4LzA3LzI2A%2BuqqQbrhKXshLwCS1QG6rOg7LKZBTEsNjM3ZALjAw9kFgJmDxUGCjIwMTgvMDcvMjYD66qpBuuhr%2BuNsAJOQwbsgqzsp4EFNywxNzdkAuQDD2QWAmYPFQYKMjAxOC8wNy8yNgPrqqkCU0sG65GQ7IKwBuusuO2VmQYxMyw0OTlkAuUDD2QWAmYPFQYKMjAxOC8wNy8yNgPrqqkCTEcG7IK87ISxBuyeoOyLpAU5LDQxNWQC5gMPZBYCZg8VBgoyMDE4LzA3LzI3A%2Bq4iAbrkZDsgrAG7ZWc7ZmUBuyeoOyLpAYxOSwwMjNkAucDD2QWAmYPFQYKMjAxOC8wNy8yNwPquIgG7IK87ISxA0tJQQbrjIDqtawFNSwyOTNkAugDD2QWAmYPFQYKMjAxOC8wNy8yNwPquIgCS1QCTEcG7IiY7JuQBTcsMTg4ZALpAw9kFgJmDxUGCjIwMTgvMDcvMjcD6riIBuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTQsMTQ4ZALqAw9kFgJmDxUGCjIwMTgvMDcvMjcD6riIAk5DAlNLBuuniOyCsAU0LDE4N2QC6wMPZBYCZg8VBgoyMDE4LzA3LzI4A%2B2GoAbrkZDsgrAG7ZWc7ZmUBuyeoOyLpAYyMiwyMDJkAuwDD2QWAmYPFQYKMjAxOC8wNy8yOAPthqAG7IK87ISxA0tJQQbrjIDqtawGMTQsMTIxZALtAw9kFgJmDxUGCjIwMTgvMDcvMjgD7YagAktUAkxHBuyImOybkAYxMywyMzBkAu4DD2QWAmYPFQYKMjAxOC8wNy8yOAPthqAG64Sl7IS8Buuhr%2BuNsAbqs6DsspkFOSwzODZkAu8DD2QWAmYPFQYKMjAxOC8wNy8yOAPthqACTkMCU0sG66eI7IKwBTYsMTAzZALwAw9kFgJmDxUGCjIwMTgvMDcvMjkD7J28BuuRkOyCsAbtlZztmZQG7J6g7IukBjIxLDA4NGQC8QMPZBYCZg8VBgoyMDE4LzA3LzI5A%2BydvAbsgrzshLEDS0lBBuuMgOq1rAYxMCw2MzFkAvIDD2QWAmYPFQYKMjAxOC8wNy8yOQPsnbwCS1QCTEcG7IiY7JuQBTksMDc5ZALzAw9kFgJmDxUGCjIwMTgvMDcvMjkD7J28BuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTcsMzU0ZAL0Aw9kFgJmDxUGCjIwMTgvMDcvMjkD7J28Ak5DAlNLBuuniOyCsAU0LDI2M2QC9QMPZBYCZg8VBgoyMDE4LzA3LzMxA%2B2ZlAbtlZztmZQCS1QG64yA7KCEBTgsMDEzZAL2Aw9kFgJmDxUGCjIwMTgvMDcvMzED7ZmUBuuRkOyCsAJMRwbsnqDsi6QGMTMsMTA3ZAL3Aw9kFgJmDxUGCjIwMTgvMDcvMzED7ZmUA0tJQQbroa%2FrjbAG6rSR7KO8BTgsNTA4ZAL4Aw9kFgJmDxUGCjIwMTgvMDcvMzED7ZmUBuyCvOyEsQJOQwbrjIDqtawFOSwwNTdkAvkDD2QWAmYPFQYKMjAxOC8wNy8zMQPtmZQCU0sG64Sl7IS8BuusuO2VmQU2LDA0OWQC%2BgMPZBYCZg8VBgoyMDE4LzA4LzAxA%2ByImAbtlZztmZQCS1QG64yA7KCEBTcsODk2ZAL7Aw9kFgJmDxUGCjIwMTgvMDgvMDED7IiYBuuRkOyCsAJMRwbsnqDsi6QGMTIsMTQ1ZAL8Aw9kFgJmDxUGCjIwMTgvMDgvMDED7IiYA0tJQQbroa%2FrjbAG6rSR7KO8BTcsMDE4ZAL9Aw9kFgJmDxUGCjIwMTgvMDgvMDED7IiYBuyCvOyEsQJOQwbrjIDqtawFOCw1MTlkAv4DD2QWAmYPFQYKMjAxOC8wOC8wMQPsiJgCU0sG64Sl7IS8BuusuO2VmQU1LDk3NmQC%2FwMPZBYCZg8VBgoyMDE4LzA4LzAyA%2BuqqQbtlZztmZQCS1QG64yA7KCEBTgsNDI1ZAKABA9kFgJmDxUGCjIwMTgvMDgvMDID66qpBuuRkOyCsAJMRwbsnqDsi6QGMTMsMzA0ZAKBBA9kFgJmDxUGCjIwMTgvMDgvMDID66qpA0tJQQbroa%2FrjbAG6rSR7KO8BTgsMzk4ZAKCBA9kFgJmDxUGCjIwMTgvMDgvMDID66qpBuyCvOyEsQJOQwbrjIDqtawFOSw0ODZkAoMED2QWAmYPFQYKMjAxOC8wOC8wMgPrqqkCU0sG64Sl7IS8BuusuO2VmQU2LDk0OGQChAQPZBYCZg8VBgoyMDE4LzA4LzA0A%2B2GoAbtlZztmZQCTkMG64yA7KCEBjEyLDE1NGQChQQPZBYCZg8VBgoyMDE4LzA4LzA0A%2B2GoANLSUEG65GQ7IKwBuq0keyjvAYxMiwyMTlkAoYED2QWAmYPFQYKMjAxOC8wOC8wNAPthqACTEcCU0sG7J6g7IukBjEyLDA3MGQChwQPZBYCZg8VBgoyMDE4LzA4LzA0A%2B2GoAbroa%2FrjbAG7IK87ISxBuyCrOyngQYyMSw2OTRkAogED2QWAmYPFQYKMjAxOC8wOC8wNAPthqACS1QG64Sl7IS8BuyImOybkAU3LDA3M2QCiQQPZBYCZg8VBgoyMDE4LzA4LzA1A%2BydvAbtlZztmZQCTkMG64yA7KCEBTgsNjE4ZAKKBA9kFgJmDxUGCjIwMTgvMDgvMDUD7J28A0tJQQbrkZDsgrAG6rSR7KO8BTgsMjgxZAKLBA9kFgJmDxUGCjIwMTgvMDgvMDUD7J28AkxHAlNLBuyeoOyLpAU4LDY4OWQCjAQPZBYCZg8VBgoyMDE4LzA4LzA1A%2BydvAbroa%2FrjbAG7IK87ISxBuyCrOyngQYxMCw4MTNkAo0ED2QWAmYPFQYKMjAxOC8wOC8wNQPsnbwCS1QG64Sl7IS8BuyImOybkAU1LDI0OWQCjgQPZBYCZg8VBgoyMDE4LzA4LzA3A%2B2ZlAbrkZDsgrAG7ZWc7ZmUBuyeoOyLpAYxMywxODRkAo8ED2QWAmYPFQYKMjAxOC8wOC8wNwPtmZQG64Sl7IS8A0tJQQbqs6DsspkGMTAsNTIwZAKQBA9kFgJmDxUGCjIwMTgvMDgvMDcD7ZmUAk5DAktUBuuniOyCsAU0LDkzNmQCkQQPZBYCZg8VBgoyMDE4LzA4LzA3A%2B2ZlAbroa%2FrjbACTEcG7Jq47IKwBTYsMDM2ZAKSBA9kFgJmDxUGCjIwMTgvMDgvMDcD7ZmUAlNLBuyCvOyEsQbrrLjtlZkFNywxMzVkApMED2QWAmYPFQYKMjAxOC8wOC8wOAPsiJgG65GQ7IKwBu2VnO2ZlAbsnqDsi6QGMTQsMDQ3ZAKUBA9kFgJmDxUGCjIwMTgvMDgvMDgD7IiYBuuEpeyEvANLSUEG6rOg7LKZBjEwLDMxMmQClQQPZBYCZg8VBgoyMDE4LzA4LzA4A%2ByImAJOQwJLVAbrp4jsgrAFNCwzODJkApYED2QWAmYPFQYKMjAxOC8wOC8wOAPsiJgG66Gv642wAkxHBuyauOyCsAYxMiwwMzhkApcED2QWAmYPFQYKMjAxOC8wOC8wOAPsiJgCU0sG7IK87ISxBuusuO2VmQU3LDE2MGQCmAQPZBYCZg8VBgoyMDE4LzA4LzA5A%2BuqqQNLSUEG66Gv642wBuq0keyjvAU2LDAyNmQCmQQPZBYCZg8VBgoyMDE4LzA4LzA5A%2BuqqQJLVAbrkZDsgrAG7IiY7JuQBTYsNTY2ZAKaBA9kFgJmDxUGCjIwMTgvMDgvMDkD66qpAk5DAlNLBuuniOyCsAU0LDMxNWQCmwQPZBYCZg8VBgoyMDE4LzA4LzA5A%2BuqqQJMRwbsgrzshLEG7J6g7IukBjEwLDAwN2QCnAQPZBYCZg8VBgoyMDE4LzA4LzA5A%2BuqqQbtlZztmZQG64Sl7IS8BuyyreyjvAU5LDQ4NWQCnQQPZBYCZg8VBgoyMDE4LzA4LzEwA%2Bq4iAJLVAbrkZDsgrAG7IiY7JuQBTgsMTE3ZAKeBA9kFgJmDxUGCjIwMTgvMDgvMTAD6riIAk5DAlNLBuuniOyCsAU1LDg2N2QCnwQPZBYCZg8VBgoyMDE4LzA4LzEwA%2Bq4iAJMRwbsgrzshLEG7J6g7IukBjEyLDcwNmQCoAQPZBYCZg8VBgoyMDE4LzA4LzEwA%2Bq4iAbtlZztmZQG64Sl7IS8BuyyreyjvAU5LDgxM2QCoQQPZBYCZg8VBgoyMDE4LzA4LzExA%2B2GoAJTSwNLSUEG66y47ZWZBjI0LDM3OWQCogQPZBYCZg8VBgoyMDE4LzA4LzExA%2B2GoAbtlZztmZQCS1QG64yA7KCEBjExLDUyOGQCowQPZBYCZg8VBgoyMDE4LzA4LzExA%2B2GoAbrhKXshLwCTEcG6rOg7LKZBjEyLDc3MWQCpAQPZBYCZg8VBgoyMDE4LzA4LzExA%2B2GoAbrkZDsgrAG66Gv642wBuyeoOyLpAYxOCw0MjZkAqUED2QWAmYPFQYKMjAxOC8wOC8xMQPthqAG7IK87ISxAk5DBuuMgOq1rAYxOCw1MzNkAqYED2QWAmYPFQYKMjAxOC8wOC8xMgPsnbwCU0sDS0lBBuusuO2VmQYxOCwzNzhkAqcED2QWAmYPFQYKMjAxOC8wOC8xMgPsnbwG7ZWc7ZmUAktUBuuMgOyghAU5LDgyN2QCqAQPZBYCZg8VBgoyMDE4LzA4LzEyA%2BydvAbrhKXshLwCTEcG6rOg7LKZBTksMjc5ZAKpBA9kFgJmDxUGCjIwMTgvMDgvMTID7J28BuuRkOyCsAbroa%2FrjbAG7J6g7IukBjE0LDE3NWQCqgQPZBYCZg8VBgoyMDE4LzA4LzEyA%2BydvAbsgrzshLECTkMG64yA6rWsBjEyLDYwM2QCqwQPZBYCZg8VBgoyMDE4LzA4LzE0A%2B2ZlAbroa%2FrjbAG7ZWc7ZmUBuyCrOyngQYxNiwzNDZkAqwED2QWAmYPFQYKMjAxOC8wOC8xNAPtmZQDS0lBAkxHBuq0keyjvAYxMCwxMzFkAq0ED2QWAmYPFQYKMjAxOC8wOC8xNAPtmZQCS1QCTkMG7IiY7JuQBTQsMzY0ZAKuBA9kFgJmDxUGCjIwMTgvMDgvMTQD7ZmUBuuRkOyCsAJTSwbsnqDsi6QGMTEsMTMyZAKvBA9kFgJmDxUGCjIwMTgvMDgvMTQD7ZmUBuyCvOyEsQbrhKXshLwG64yA6rWsBjExLDE0NmQCsAQPZBYCZg8VBgoyMDE4LzA4LzE1A%2ByImANLSUECTEcG6rSR7KO8BjExLDUyMGQCsQQPZBYCZg8VBgoyMDE4LzA4LzE1A%2ByImAJLVAJOQwbsiJjsm5AFNiw3NTJkArIED2QWAmYPFQYKMjAxOC8wOC8xNQPsiJgG65GQ7IKwAlNLBuyeoOyLpAYxMiwyNDRkArMED2QWAmYPFQYKMjAxOC8wOC8xNQPsiJgG7IK87ISxBuuEpeyEvAbrjIDqtawGMTIsMDQ4ZAK0BA9kFgJmDxUGCjIwMTgvMDgvMTYD66qpBuyCvOyEsQbtlZztmZQG64yA6rWsBTYsODA5ZAK1BA9kFgJmDxUGCjIwMTgvMDgvMTYD66qpBuuhr%2BuNsANLSUEG7IKs7KeBBjEwLDgwMGQCtgQPZBYCZg8VBgoyMDE4LzA4LzE2A%2BuqqQJTSwJMRwbrrLjtlZkFOCwzODhkArcED2QWAmYPFQYKMjAxOC8wOC8xNgPrqqkCS1QCTkMG7IiY7JuQBTMsMzU5ZAK4BA9kFgJmDxUGCjIwMTgvMDgvMTYD66qpBuuRkOyCsAbrhKXshLwG7J6g7IukBTksMTY4ZAK5BA9kFgJmDxUGCjIwMTgvMDkvMDQD7ZmUBuuRkOyCsANLSUEG7J6g7IukBjE0LDYwM2QCugQPZBYCZg8VBgoyMDE4LzA5LzA0A%2B2ZlAJLVAJMRwbsiJjsm5AFNCwyODdkArsED2QWAmYPFQYKMjAxOC8wOS8wNAPtmZQG7ZWc7ZmUBuuhr%2BuNsAbrjIDsoIQFNywyODVkArwED2QWAmYPFQYKMjAxOC8wOS8wNAPtmZQCTkMG7IK87ISxBuuniOyCsAU0LDI1OGQCvQQPZBYCZg8VBgoyMDE4LzA5LzA0A%2B2ZlAJTSwbrhKXshLwG66y47ZWZBTUsMDAyZAK%2BBA9kFgJmDxUGCjIwMTgvMDkvMDUD7IiYBuuRkOyCsANLSUEG7J6g7IukBjE3LDE5MWQCvwQPZBYCZg8VBgoyMDE4LzA5LzA1A%2ByImAJLVAJMRwbsiJjsm5AFNCwyNTFkAsAED2QWAmYPFQYKMjAxOC8wOS8wNQPsiJgG7ZWc7ZmUBuuhr%2BuNsAbrjIDsoIQFNyw5MTFkAsEED2QWAmYPFQYKMjAxOC8wOS8wNQPsiJgCTkMG7IK87ISxBuuniOyCsAU1LDEzMmQCwgQPZBYCZg8VBgoyMDE4LzA5LzA1A%2ByImAJTSwbrhKXshLwG66y47ZWZBTUsMjQxZALDBA9kFgJmDxUGCjIwMTgvMDkvMDYD66qpAktUBu2VnO2ZlAbsiJjsm5AFNywyNjhkAsQED2QWAmYPFQYKMjAxOC8wOS8wNgPrqqkCTEcCTkMG7J6g7IukBTYsOTQ0ZALFBA9kFgJmDxUGCjIwMTgvMDkvMDYD66qpBuyCvOyEsQbrkZDsgrAG64yA6rWsBTYsNTE3ZALGBA9kFgJmDxUGCjIwMTgvMDkvMDYD66qpBuuhr%2BuNsAJTSwbsmrjsgrAFNCw2NjhkAscED2QWAmYPFQYKMjAxOC8wOS8wNgPrqqkDS0lBBuuEpeyEvAbqtJHso7wFNiwzOTdkAsgED2QWAmYPFQYKMjAxOC8wOS8wNwPquIgCS1QG7ZWc7ZmUBuyImOybkAU5LDkzNmQCyQQPZBYCZg8VBgoyMDE4LzA5LzA3A%2Bq4iAJMRwJOQwbsnqDsi6QFNyw5MTdkAsoED2QWAmYPFQYKMjAxOC8wOS8wNwPquIgG7IK87ISxBuuRkOyCsAbrjIDqtawGMTEsMDc4ZALLBA9kFgJmDxUGCjIwMTgvMDkvMDcD6riIBuuhr%2BuNsAJTSwbsmrjsgrAFNCwyNDBkAswED2QWAmYPFQYKMjAxOC8wOS8wNwPquIgDS0lBBuuEpeyEvAbqtJHso7wFNiw3NzFkAs0ED2QWAmYPFQYKMjAxOC8wOS8wOAPthqACTEcG7ZWc7ZmUBuyeoOyLpAYyMiw5MDhkAs4ED2QWAmYPFQYKMjAxOC8wOS8wOAPthqAG64Sl7IS8AktUBuqzoOyymQUzLDA5M2QCzwQPZBYCZg8VBgoyMDE4LzA5LzA4A%2B2GoAJOQwbroa%2FrjbAG66eI7IKwBTgsNjM3ZALQBA9kFgJmDxUGCjIwMTgvMDkvMDgD7YagAlNLBuuRkOyCsAbrrLjtlZkGMjQsMDg0ZALRBA9kFgJmDxUGCjIwMTgvMDkvMDgD7YagA0tJQQbsgrzshLEG6rSR7KO8BjEzLDc5NmQC0gQPZBYCZg8VBgoyMDE4LzA5LzA5A%2BydvAJMRwbtlZztmZQG7J6g7IukBjE5LDI4NmQC0wQPZBYCZg8VBgoyMDE4LzA5LzA5A%2BydvAbrhKXshLwCS1QG6rOg7LKZBTIsNDczZALUBA9kFgJmDxUGCjIwMTgvMDkvMDkD7J28Ak5DBuuhr%2BuNsAbrp4jsgrAFOCwwOTFkAtUED2QWAmYPFQYKMjAxOC8wOS8wOQPsnbwCU0sG65GQ7IKwBuusuO2VmQYyMCw0OThkAtYED2QWAmYPFQYKMjAxOC8wOS8wOQPsnbwDS0lBBuyCvOyEsQbqtJHso7wGMTEsMzUyZALXBA9kFgJmDxUGCjIwMTgvMDkvMTED7ZmUBuyCvOyEsQbtlZztmZQG64yA6rWsBTYsMTQxZALYBA9kFgJmDxUGCjIwMTgvMDkvMTED7ZmUAk5DA0tJQQbrp4jsgrAFNCwwNzdkAtkED2QWAmYPFQYKMjAxOC8wOS8xMQPtmZQCU0sCS1QG66y47ZWZBTQsOTkzZALaBA9kFgJmDxUGCjIwMTgvMDkvMTED7ZmUBuuhr%2BuNsAbrkZDsgrAG7IKs7KeBBTgsOTkwZALbBA9kFgJmDxUGCjIwMTgvMDkvMTED7ZmUAkxHBuuEpeyEvAbsnqDsi6QFOCwzODZkAtwED2QWAmYPFQYKMjAxOC8wOS8xMgPsiJgG7IK87ISxBu2VnO2ZlAbrjIDqtawFOCw2NzlkAt0ED2QWAmYPFQYKMjAxOC8wOS8xMgPsiJgCTkMDS0lBBuuniOyCsAU0LDA4NmQC3gQPZBYCZg8VBgoyMDE4LzA5LzEyA%2ByImAJTSwJLVAbrrLjtlZkFNCwyNTRkAt8ED2QWAmYPFQYKMjAxOC8wOS8xMgPsiJgG66Gv642wBuuRkOyCsAbsgqzsp4EFNywwMjFkAuAED2QWAmYPFQYKMjAxOC8wOS8xMgPsiJgCTEcG64Sl7IS8BuyeoOyLpAU4LDQ2M2QC4QQPZBYCZg8VBgoyMDE4LzA5LzEzA%2BuqqQbrkZDsgrACS1QG7J6g7IukBTksMjE0ZALiBA9kFgJmDxUGCjIwMTgvMDkvMTMD66qpBuyCvOyEsQJMRwbrjIDqtawFOCwxMDRkAuMED2QWAmYPFQYKMjAxOC8wOS8xMwPrqqkG7ZWc7ZmUAlNLBuyyreyjvAU5LDQ1NGQC5AQPZBYCZg8VBgoyMDE4LzA5LzE0A%2Bq4iAbroa%2FrjbADS0lBBuyCrOyngQU1LDIzMWQC5QQPZBYCZg8VBgoyMDE4LzA5LzE0A%2Bq4iAbrkZDsgrACS1QG7J6g7IukBjEwLDEyNGQC5gQPZBYCZg8VBgoyMDE4LzA5LzE0A%2Bq4iAbsgrzshLECTEcG64yA6rWsBTUsNTI4ZALnBA9kFgJmDxUGCjIwMTgvMDkvMTQD6riIBu2VnO2ZlAJTSwbssq3so7wFOCwzODFkAugED2QWAmYPFQYKMjAxOC8wOS8xNAPquIgCTkMG64Sl7IS8BuuniOyCsAUzLDEwN2QC6QQPZBYCZg8VBgoyMDE4LzA5LzE1A%2B2GoAbtlZztmZQCTEcG64yA7KCEBjEyLDcxNWQC6gQPZBYCZg8VBgoyMDE4LzA5LzE1A%2B2GoAbrkZDsgrACTkMG7J6g7IukBjE2LDU1OGQC6wQPZBYCZg8VBgoyMDE4LzA5LzE1A%2B2GoANLSUECU0sG6rSR7KO8BjExLDQ4MGQC7AQPZBYCZg8VBgoyMDE4LzA5LzE1A%2B2GoAJLVAbsgrzshLEG7IiY7JuQBjExLDcyNGQC7QQPZBYCZg8VBgoyMDE4LzA5LzE1A%2B2GoAbroa%2FrjbAG64Sl7IS8BuyCrOyngQYyNSwwMDBkAu4ED2QWAmYPFQYKMjAxOC8wOS8xNgPsnbwG7ZWc7ZmUAkxHBuuMgOyghAYxMiwwMTJkAu8ED2QWAmYPFQYKMjAxOC8wOS8xNgPsnbwG65GQ7IKwAk5DBuyeoOyLpAYxMywyMTRkAvAED2QWAmYPFQYKMjAxOC8wOS8xNgPsnbwDS0lBAlNLBuq0keyjvAU4LDMwNmQC8QQPZBYCZg8VBgoyMDE4LzA5LzE2A%2BydvAJLVAbsgrzshLEG7IiY7JuQBjEyLDEwMGQC8gQPZBYCZg8VBgoyMDE4LzA5LzE2A%2BydvAbroa%2FrjbAG64Sl7IS8BuyCrOyngQU5LDA2MWQC8wQPZBYCZg8VBgoyMDE4LzA5LzE4A%2B2ZlAJOQwbtlZztmZQG66eI7IKwBTQsNDM0ZAL0BA9kFgJmDxUGCjIwMTgvMDkvMTgD7ZmUBuyCvOyEsQNLSUEG64yA6rWsBTUsNzMzZAL1BA9kFgJmDxUGCjIwMTgvMDkvMTgD7ZmUAkxHBuuhr%2BuNsAbsnqDsi6QGMTEsMTAxZAL2BA9kFgJmDxUGCjIwMTgvMDkvMTgD7ZmUBuuEpeyEvAbrkZDsgrAG6rOg7LKZBTQsNzQzZAL3BA9kFgJmDxUGCjIwMTgvMDkvMTgD7ZmUAktUAlNLBuyImOybkAU0LDA1MGQC%2BAQPZBYCZg8VBgoyMDE4LzA5LzE5A%2ByImAJOQwbtlZztmZQG66eI7IKwBTQsNTU2ZAL5BA9kFgJmDxUGCjIwMTgvMDkvMTkD7IiYBuyCvOyEsQNLSUEG64yA6rWsBTksMDY3ZAL6BA9kFgJmDxUGCjIwMTgvMDkvMTkD7IiYAkxHBuuhr%2BuNsAbsnqDsi6QGMTAsMTI2ZAL7BA9kFgJmDxUGCjIwMTgvMDkvMTkD7IiYBuuEpeyEvAbrkZDsgrAG6rOg7LKZBTQsNDcwZAL8BA9kFgJmDxUGCjIwMTgvMDkvMTkD7IiYAktUAlNLBuyImOybkAUzLDkxMGQC%2FQQPZBYCZg8VBgoyMDE4LzA5LzIwA%2BuqqQJTSwbtlZztmZQG66y47ZWZBjEyLDE2MGQC%2FgQPZBYCZg8VBgoyMDE4LzA5LzIwA%2BuqqQbroa%2FrjbACS1QG7IKs7KeBBTMsNzA5ZAL%2FBA9kFgJmDxUGCjIwMTgvMDkvMjAD66qpA0tJQQJOQwbqtJHso7wFOCwyOTNkAoAFD2QWAmYPFQYKMjAxOC8wOS8yMAPrqqkCTEcG65GQ7IKwBuyeoOyLpAYxMiwwMTlkAoEFD2QWAmYPFQYKMjAxOC8wOS8yMAPrqqkG64Sl7IS8BuyCvOyEsQbqs6DsspkFMiw3OTNkAoIFD2QWAmYPFQYKMjAxOC8wOS8yMQPquIgCU0sG7ZWc7ZmUBuusuO2VmQYxNywxNTZkAoMFD2QWAmYPFQYKMjAxOC8wOS8yMQPquIgDS0lBAk5DBuq0keyjvAYxMCwxMTRkAoQFD2QWAmYPFQYKMjAxOC8wOS8yMQPquIgCTEcG65GQ7IKwBuyeoOyLpAYxNywwMzFkAoUFD2QWAmYPFQYKMjAxOC8wOS8yMQPquIgG64Sl7IS8BuyCvOyEsQbqs6DsspkFMywwMTNkAoYFD2QWAmYPFQYKMjAxOC8wOS8yMgPthqAG7ZWc7ZmUA0tJQQbrjIDsoIQGMTMsMDAwZAKHBQ9kFgJmDxUGCjIwMTgvMDkvMjID7YagAkxHAktUBuyeoOyLpAYxMywzMTRkAogFD2QWAmYPFQYKMjAxOC8wOS8yMgPthqAG7IK87ISxBuuhr%2BuNsAbrjIDqtawGMTgsNDYxZAKJBQ9kFgJmDxUGCjIwMTgvMDkvMjID7YagAk5DBuuRkOyCsAbrp4jsgrAFOCwxODJkAooFD2QWAmYPFQYKMjAxOC8wOS8yMgPthqAG64Sl7IS8AlNLBuqzoOyymQU1LDQyNmQCiwUPZBYCZg8VBgoyMDE4LzA5LzIzA%2BydvAbtlZztmZQDS0lBBuuMgOyghAYxMCwxMjBkAowFD2QWAmYPFQYKMjAxOC8wOS8yMwPsnbwCTEcCS1QG7J6g7IukBTksNDQ0ZAKNBQ9kFgJmDxUGCjIwMTgvMDkvMjMD7J28BuyCvOyEsQbroa%2FrjbAG64yA6rWsBjEyLDgzNWQCjgUPZBYCZg8VBgoyMDE4LzA5LzIzA%2BydvAJOQwbrkZDsgrAG66eI7IKwBTYsMDM1ZAKPBQ9kFgJmDxUGCjIwMTgvMDkvMjMD7J28BuuEpeyEvAJTSwbqs6DsspkFMyw1NDlkApAFD2QWAmYPFQYKMjAxOC8wOS8yNQPtmZQCS1QDS0lBBuyImOybkAYyMCwwMDZkApEFD2QWAmYPFQYKMjAxOC8wOS8yNQPtmZQCU0sCTEcG66y47ZWZBjIyLDY1NWQCkgUPZBYCZg8VBgoyMDE4LzA5LzI1A%2B2ZlAbroa%2FrjbACTkMG7IKs7KeBBjEzLDg1MGQCkwUPZBYCZg8VBgoyMDE4LzA5LzI1A%2B2ZlAbtlZztmZQG7IK87ISxBuuMgOyghAYxMywwMDBkApQFD2QWAmYPFQYKMjAxOC8wOS8yNQPtmZQG65GQ7IKwBuuEpeyEvAbsnqDsi6QGMjQsNTUzZAKVBQ9kFgJmDxUGCjIwMTgvMDkvMjYD7IiYAktUA0tJQQbsiJjsm5AGMjAsMjI2ZAKWBQ9kFgJmDxUGCjIwMTgvMDkvMjYD7IiYAlNLAkxHBuusuO2VmQYxOSw5NzRkApcFD2QWAmYPFQYKMjAxOC8wOS8yNgPsiJgG66Gv642wAk5DBuyCrOyngQYxMCwyNTNkApgFD2QWAmYPFQYKMjAxOC8wOS8yNgPsiJgG7ZWc7ZmUBuyCvOyEsQbrjIDsoIQGMTEsNjM3ZAKZBQ9kFgJmDxUGCjIwMTgvMDkvMjYD7IiYBuuRkOyCsAbrhKXshLwG7J6g7IukBjE1LDIzMmQCmgUPZBYCZg8VBgoyMDE4LzA5LzI3A%2BuqqQJMRwNLSUEG7J6g7IukBjE3LDkzN2QCmwUPZBYCZg8VBgoyMDE4LzA5LzI3A%2BuqqQbsgrzshLECS1QG64yA6rWsBTUsNzkyZAKcBQ9kFgJmDxUGCjIwMTgvMDkvMjcD66qpBuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTMsMzgzZAKdBQ9kFgJmDxUGCjIwMTgvMDkvMjcD66qpAlNLAk5DBuusuO2VmQU3LDM5OGQCngUPZBYCZg8VBgoyMDE4LzA5LzI3A%2BuqqQbtlZztmZQG65GQ7IKwBuuMgOyghAU4LDEwOWQCnwUPZBYCZg8VBgoyMDE4LzA5LzI4A%2Bq4iAJMRwNLSUEG7J6g7IukBjIxLDc2NmQCoAUPZBYCZg8VBgoyMDE4LzA5LzI4A%2Bq4iAbsgrzshLECS1QG64yA6rWsBTksNjgzZAKhBQ9kFgJmDxUGCjIwMTgvMDkvMjgD6riIBuuEpeyEvAbroa%2FrjbAG6rOg7LKZBTMsNjU5ZAKiBQ9kFgJmDxUGCjIwMTgvMDkvMjgD6riIAlNLAk5DBuusuO2VmQU2LDc0NmQCowUPZBYCZg8VBgoyMDE4LzA5LzI4A%2Bq4iAbtlZztmZQG65GQ7IKwBuuMgOyghAYxMCw3MDVkAqQFD2QWAmYPFQYKMjAxOC8wOS8yOQPthqADS0lBBu2VnO2ZlAbqtJHso7wGMTcsODEyZAKlBQ9kFgJmDxUGCjIwMTgvMDkvMjkD7YagBuuRkOyCsAJMRwbsnqDsi6QGMjQsMTgzZAKmBQ9kFgJmDxUGCjIwMTgvMDkvMjkD7YagAktUBuuhr%2BuNsAbsiJjsm5AGMTQsMjkwZAKnBQ9kFgJmDxUGCjIwMTgvMDkvMjkD7YagBuuEpeyEvAJOQwbqs6DsspkFMyw2MDNkAqgFD2QWAmYPFQYKMjAxOC8wOS8yOQPthqAG7IK87ISxAlNLBuuMgOq1rAYxOCwyMjBkAqkFD2QWAmYPFQYKMjAxOC8wOS8zMAPsnbwDS0lBBu2VnO2ZlAbqtJHso7wGMTYsMTM3ZAKqBQ9kFgJmDxUGCjIwMTgvMDkvMzAD7J28BuuRkOyCsAJMRwbsnqDsi6QGMTksMjc0ZAKrBQ9kFgJmDxUGCjIwMTgvMDkvMzAD7J28AktUBuuhr%2BuNsAbsiJjsm5AGMTEsOTgzZAKsBQ9kFgJmDxUGCjIwMTgvMDkvMzAD7J28BuuEpeyEvAJOQwbqs6DsspkFNCw1NDJkAq0FD2QWAmYPFQYKMjAxOC8wOS8zMAPsnbwG7IK87ISxAlNLBuuMgOq1rAYxNCw2NTFkAq4FD2QWAmYPFQYKMjAxOC8xMC8wMgPtmZQCTEcCS1QG7J6g7IukBTUsNTEyZAKvBQ9kFgJmDxUGCjIwMTgvMTAvMDID7ZmUAlNLBuuhr%2BuNsAbrrLjtlZkGMTMsNzgyZAKwBQ9kFgJmDxUGCjIwMTgvMTAvMDID7ZmUA0tJQQJOQwbqtJHso7wFOCw4ODZkArEFD2QWAmYPFQYKMjAxOC8xMC8wMwPsiJgG7IK87ISxA0tJQQbrjIDqtawGMTksNDMyZAKyBQ9kFgJmDxUGCjIwMTgvMTAvMDMD7IiYAkxHAktUBuyeoOyLpAYxMSw4OTZkArMFD2QWAmYPFQYKMjAxOC8xMC8wMwPsiJgG7ZWc7ZmUBuuhr%2BuNsAbrjIDsoIQGMTMsMDAwZAK0BQ9kFgJmDxUGCjIwMTgvMTAvMDQD66qpAlNLA0tJQQbrrLjtlZkGMTMsNTU2ZAK1BQ9kFgJmDxUGCjIwMTgvMTAvMDQD66qpBu2VnO2ZlAbroa%2FrjbAG64yA7KCEBTksNTU1ZAK2BQ9kFgJmDxUGCjIwMTgvMTAvMDYD7YagBuuhr%2BuNsAbtlZztmZQG7IKs7KeBBTksNDQ5ZAK3BQ9kFgJmDxUGCjIwMTgvMTAvMDYD7YagAlNLA0tJQQbrrLjtlZkFOSwyNDFkArgFD2QWAmYPFQYKMjAxOC8xMC8wNgPthqACU0sDS0lBBuusuO2VmQYyMiwwMDJkArkFD2QWAmYPFQYKMjAxOC8xMC8wNgPthqAG65GQ7IKwAkxHBuyeoOyLpAYxOSw4OTFkAroFD2QWAmYPFQYKMjAxOC8xMC8wNgPthqACS1QG7IK87ISxBuyImOybkAU4LDI5MWQCuwUPZBYCZg8VBgoyMDE4LzEwLzA2A%2B2GoAJOQwbrhKXshLwG66eI7IKwBTUsMjk3ZAK8BQ9kFgJmDxUGCjIwMTgvMTAvMDcD7J28BuuRkOyCsANLSUEG7J6g7IukBjI1LDAwMGQCvQUPZBYCZg8VBgoyMDE4LzEwLzA3A%2BydvAJOQwbroa%2FrjbAG66eI7IKwBjExLDAwMGQCvgUPZBYCZg8VBgoyMDE4LzEwLzA5A%2B2ZlAJLVAbtlZztmZQG7IiY7JuQBjIwLDUxMmQCvwUPZBYCZg8VBgoyMDE4LzEwLzA5A%2B2ZlAbroa%2FrjbADS0lBBuyCrOyngQYyNSwwMDBkAsAFD2QWAmYPFQYKMjAxOC8xMC8wOQPtmZQCU0sG7IK87ISxBuusuO2VmQYyMiwwMzFkAsEFD2QWAmYPFQYKMjAxOC8xMC8xMAPsiJgDS0lBBu2VnO2ZlAbqtJHso7wFNiwxMzNkAsIFD2QWAmYPFQYKMjAxOC8xMC8xMAPsiJgG66Gv642wAktUBuyCrOyngQU0LDc5M2QCwwUPZBYCZg8VBgoyMDE4LzEwLzEwA%2ByImAbroa%2FrjbACS1QG7IKs7KeBBjExLDUyOGQCxAUPZBYCZg8VBgoyMDE4LzEwLzEwA%2ByImAbrkZDsgrACU0sG7J6g7IukBjExLDEyNmQCxQUPZBYCZg8VBgoyMDE4LzEwLzExA%2BuqqQNLSUEG66Gv642wBuq0keyjvAU3LDc0NWQCxgUPZBYCZg8VBgoyMDE4LzEwLzExA%2BuqqQbrkZDsgrACU0sG7J6g7IukBjEwLDk4N2QCxwUPZBYCZg8VBgoyMDE4LzEwLzEyA%2Bq4iANLSUEG66Gv642wBuq0keyjvAYxMiwzMThkAsgFD2QWAmYPFQYKMjAxOC8xMC8xMgPquIgG65GQ7IKwAk5DBuyeoOyLpAYxMCw5MTRkAskFD2QWAmYPFQYKMjAxOC8xMC8xMgPquIgCS1QG64Sl7IS8BuyImOybkAYxMCwyNjhkAsoFD2QWAmYPFQYKMjAxOC8xMC8xMwPthqAG65GQ7IKwAktUBuyeoOyLpAYxOCw1MjhkAssFD2QWAmYPFQYKMjAxOC8xMC8xMwPthqACU0sCTEcG66y47ZWZBjIyLDM5OWQCzAUPZBYCZg8VBgoyMDE4LzEwLzEzA%2B2GoANLSUEG66Gv642wBuq0keyjvAYxNSw2NDBkAs0FD2QWAmYPFQYKMjAxOC8xMC8xMwPthqAG7ZWc7ZmUAk5DBuuMgOyghAYxMywwMDBkAs4FD2QWAmYPFQYKMjAxOC8xMC8xMwPthqAG7IK87ISxBuuEpeyEvAbrjIDqtawGMjQsMDAwZALPBQ9kFgJmDxUGCjIwMTgvMTAvMTQD7J28Buuhr%2BuNsAbrkZDsgrAG7IKs7KeBBTYsNDE0ZGTdMZk%2BjDXy%2FW8XnWCNIcLf45YnlVYTxebpT4agZalLdw%3D%3D&__VIEWSTATEGENERATOR=652934D0&__EVENTVALIDATION=%2FwEdADXU9%2FPEBdAAFebDaXCvrNIaHiWnp0LNW1CovBvF53tIGamt6aSj%2Fm76nU19As1b2a1LvR0m0U%2FcGecKHUJyo%2BZww7igLzDrq4UG%2FNjF45Epc2Svq5Xu2ww%2Frbf5rUuNc9tABwcPP9Ar66p3O99YlFi%2BEYaK7r3qKkM8qndrvEXS3TlqoYriLmcOg65Blyfex3Y176rDDf4%2FW3CjVlHylsjUSg%2F4wNdMrTThzDM2XKftdmzYy1d%2F6KHjC2Wz37%2FjENY6L9h8Nh2Aew01s8nmaiAHFlPZpzkvyNTiz7LttPHRPht303lJks0cqO7BESB1ojZDYHgUmzSiynCk4PNhxmeT%2BkQ8AqPLE4RrhO6ZFCqeBCxU8kSoyaew%2B8cH10gkal%2FmFdUWslBO5btpZyMqdhF4dCA%2Bbt9lrvQ7Bnt3sOnVYFVlRfF3%2B6ImEs0Rr71fGaerjKbYPJCO%2B0m1OPvueZTZrs%2FMpg0cmfWLHjUb0iE5C4lcepFRdFRmSUFma4Vamw0BG%2FPf3I75C3rKX8xVtEwoJY%2FImRWG9e44Fizmqo6gKWu2meKm3KVppYH3GHNwNVFPYtlfmO3Lf4yx8HHxvftGGtE4T6HJpDMlYJEUS25kk1Q5M9Telw7HPRYTg%2BxOw86Sx25ZHoeJgwkPE5GABMcLtg3yNj35r%2FXfPRrtwZQUuBupjLd9S%2BDuUtok8FEYwvrh85%2Fdo%2FIaGwAfWyg1Ca8g6vs6A6bvi8OgaKhmA9wYvg%2BoJnlVLRq9xaGP3AzvRqGkqjm3VVr6gvfxMdwvJ7cc5oKiLPMOBwXuH%2BObXRx7JshvbdKtGXi9oQ2N8duOuv8IFnL9dgI0y8z%2F1CGrordsB2bZ7SwT8Vk2KPyF9%2B8UwkqTYbbg5LSTZpFuphH4bWviPY5lT4SFb6LfGYEmX%2F3clgnduBwqWqzWM7tUHwqLWf6NjF2BjjDcRXOhkhBbGcMTpikaIZWdCf92C8WZwMVtwNIggge%2FiBsmHXbiFqA6%2FgFOQbkutVsAoy4k1md6XmaAnYGaa%2FqFz5URurwExDg1YkqjXUITQSbe9NY3vLuOxt8wTiEmXdRxa3qD588OiAwhNHHziCWD4TcILjGYDLoIuuwNcLXb9Hf3j9q4Bnx0xaGIMfHi4eR%2FRl3Ui5co6IVi1GNz1xtQs%2BxeT1Rqjtqe&__ASYNCPOST=true&ctl00%24ctl00%24ctl00%24cphContents%24cphContents%24cphContents%24btnSearch=%EA%B2%80%EC%83%89"

headers = {
    'sec-fetch-mode': "cors",
    'origin': "https://www.koreabaseball.com",
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache",
    'x-requested-with': "XMLHttpRequest",
    'x-microsoftajax': "Delta=true",
    'accept': "*/*",
    'postman-token': "747dff74-a24b-acc8-6066-2d79fd6b99b9"
    }

conn.request("POST", "/History/Crowd/GraphDaily.aspx", payload, headers)

# Request의 응답은 getresponse() 함수를 통해 받을 수 있다
res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

1|#||4|138371|updatePanel|cphContents_cphContents_cphContents_udpRecord|
				<div class="tab-depth2">
					<ul class="tab">
						<li><a href="/History/Crowd/GraphTeam.aspx">구단별관중현황</a></li>
						<li class="on"><a href="/History/Crowd/GraphDaily.aspx">일자별관중현황</a></li>
						<li><a href="/History/Crowd/GraphYear.aspx">연도별관중현황</a></li>
					</ul>
				</div>
				<div class="compare schItem">
					<strong class="title">검색항목</strong>
                    <select name="ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlSeason" id="cphContents_cphContents_cphContents_ddlSeason" class="select02">
	<option value="2019">시즌별</option>
	<option value="2017">2017</option>
	<option selected="selected" value="2018">2018</option>
	<option value="2019">2019</option>

</select>
					<select name="ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlMonth" id="cphContents_cphContents_cphContents_ddlMonth" class="select02">
	<option selected="selected" value="0">월별</option>
	<option value="3">03월

In [123]:
df_baseball_2018 = pd.read_html(data.decode("utf-8"))[0]
df_baseball_2018.head()

,날짜,요일,홈,방문,구장,관중수
0,2018/03/24,토,넥센,한화,고척,15055
1,2018/03/24,토,KIA,KT,광주,20500
2,2018/03/24,토,NC,LG,마산,11000
3,2018/03/24,토,SK,롯데,문학,25000
4,2018/03/24,토,두산,삼성,잠실,25000


#### 두 번째 대안 : 개발자 도구

 - 표의 아무 곳에 마우스 우클릭 -> 검사
 - Elements 탭에, 페이지를 구성하는 HTML 태그들을 확인할 수 있다.
 - Table 태그를 찾아 우클릭 -> Copy -> Copy element
 - 텍스트 에디터에 붙여넣기를 하면, 클립보드에 저장된 테이블 내용을 확인할 수 있다.
 - 붙여넣기 한 내용을 .html 확장자로 저장!
    

In [124]:
# read_html 함수는 항상 DataFrame의 list를 반환하는것에 유의
df_baseball_2017 = pd.read_html('data/baseball_2017.html', encoding='utf-8')[0]
print(df_baseball_2017)

                                            날짜  \
0                                   2017/03/31   
1                                   2017/03/31   
2                                   2017/03/31   
3                                   2017/03/31   
4                                   2017/03/31   
5                                   2017/04/01   
6                                   2017/04/01   
7                                   2017/04/01   
8                                   2017/04/01   
9                                   2017/04/01   
10                                  2017/04/02   
11                                  2017/04/02   
12                                  2017/04/02   
13                                  2017/04/02   
14                                  2017/04/02   
15                                  2017/04/04   
16                                  2017/04/04   
17                                  2017/04/04   
18                                  2017/04/04   


In [125]:
df_baseball_2018 = pd.read_html('data/baseball_2018.html', encoding='utf-8')[0]
print(df_baseball_2018)

                                            날짜  \
0                                   2018/03/24   
1                                   2018/03/24   
2                                   2018/03/24   
3                                   2018/03/24   
4                                   2018/03/24   
5                                   2018/03/25   
6                                   2018/03/25   
7                                   2018/03/25   
8                                   2018/03/25   
9                                   2018/03/25   
10                                  2018/03/27   
11                                  2018/03/27   
12                                  2018/03/27   
13                                  2018/03/27   
14                                  2018/03/27   
15                                  2018/03/28   
16                                  2018/03/28   
17                                  2018/03/28   
18                                  2018/03/28   


---
### 날씨 데이터 수집하기

 - Google "날씨 데이터" -> 기상자료개방포털 -> 방재기상관측 -> 파일셋 조회
 - 지도에서 각 구장과 가장 가까운 관측장치를 선택
 - 자료 형태는 시간 자료
 - 기간은 2017년 ~ 2018년

수집된 파일은 SURFACE_AWS_관측기ID_HR_시작연도_종료연도_생산연도.csv 의 형태로 저장된다.
10개의 관측소 * 2개년도 = 총 20개의 파일

#### <실습>
 - 파일을 하나씩 불러와서 DataFrame에 저장하고
 - df_weather_list에 추가하기

In [126]:
# 파일 불러오기

import glob

df_weather_list = []
for each_file in glob.glob("data/SURFACE_*.csv"):
    # print(each_file)
    df = pd.read_csv(each_file, encoding = "EUC_KR")
    df_weather_list.append(df)

print(df_weather_list)

[       지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
0     423  2017-01-01 00:00     0.2     35.3      0.6      0.0        NaN   
1     423  2017-01-01 01:00     0.2      8.4      0.5      0.0        NaN   
2     423  2017-01-01 02:00    -0.3      0.0      0.1      0.0        NaN   
3     423  2017-01-01 03:00    -0.4      0.0      0.0      0.0        NaN   
4     423  2017-01-01 04:00    -0.7      0.0      0.2      0.0        NaN   
5     423  2017-01-01 05:00    -0.6      0.0      0.0      0.0        NaN   
6     423  2017-01-01 06:00    -1.1    350.1      0.5      0.0        NaN   
7     423  2017-01-01 07:00    -0.4      0.0      0.0      0.0        NaN   
8     423  2017-01-01 08:00    -0.8    219.7      0.6      0.0        NaN   
9     423  2017-01-01 09:00     0.9     91.4      0.7      0.0        NaN   
10    423  2017-01-01 10:00     2.3      0.0      0.2      0.0        NaN   
11    423  2017-01-01 11:00     4.2    119.5      1.0      0.0        NaN  

---
## 3. 데이터 살펴보기 및 준비하기


#### <실습>
 - 2017년, 2018년 관중수 데이터의 첫 7줄과 마지막 7줄을 출력해보자

In [127]:
df_baseball_2017.head(7)

,날짜,요일,홈,방문,구장,관중수
0,2017/03/31,금,두산,한화,잠실,21121
1,2017/03/31,금,삼성,KIA,대구,13505
2,2017/03/31,금,SK,KT,문학,13649
3,2017/03/31,금,넥센,LG,고척,8013
4,2017/03/31,금,NC,롯데,마산,11000
5,2017/04/01,토,두산,한화,잠실,20636
6,2017/04/01,토,삼성,KIA,대구,10062


In [128]:
df_baseball_2017.tail(7)

,날짜,요일,홈,방문,구장,관중수
714,2017/10/02,월,KT,KIA,수원,22000
715,2017/10/03,화,KT,KIA,수원,22000
716,2017/10/03,화,롯데,LG,사직,26600
717,2017/10/03,화,한화,NC,대전,10401
718,2017/10/03,화,두산,SK,잠실,25000
719,2017/10/03,화,삼성,넥센,대구,24000
720,"경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688","경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688","경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688","경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688","경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688","경기수 : 720 경기평균 : 11,668 경기 합계 : 8,400,688"


In [129]:
df_baseball_2018.head(7)

,날짜,요일,홈,방문,구장,관중수
0,2018/03/24,토,넥센,한화,고척,15055
1,2018/03/24,토,KIA,KT,광주,20500
2,2018/03/24,토,NC,LG,마산,11000
3,2018/03/24,토,SK,롯데,문학,25000
4,2018/03/24,토,두산,삼성,잠실,25000
5,2018/03/25,일,넥센,한화,고척,13010
6,2018/03/25,일,KIA,KT,광주,20500


In [130]:
df_baseball_2018.tail(7)

,날짜,요일,홈,방문,구장,관중수
714,2018/10/13,토,두산,KT,잠실,18528
715,2018/10/13,토,SK,LG,문학,22399
716,2018/10/13,토,KIA,롯데,광주,15640
717,2018/10/13,토,한화,NC,대전,13000
718,2018/10/13,토,삼성,넥센,대구,24000
719,2018/10/14,일,롯데,두산,사직,6414
720,"경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742","경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742","경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742","경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742","경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742","경기수 : 720 경기평균 : 11,214 경기 합계 : 8,073,742"


#### <실습>
 - 각 DataFrame의 마지막 Row에 있는 데이터를 삭제해보자

In [131]:
df_baseball_2017 = df_baseball_2017.head(-1)
#df_baseball_2017 = df_baseball_2017.iloc[:-1]
#df_baseball_2017 = df_baseball_2017.drop(len(df_baseball_2017)-1,axis=0)
df_baseball_2017.tail()

,날짜,요일,홈,방문,구장,관중수
715,2017/10/03,화,KT,KIA,수원,22000
716,2017/10/03,화,롯데,LG,사직,26600
717,2017/10/03,화,한화,NC,대전,10401
718,2017/10/03,화,두산,SK,잠실,25000
719,2017/10/03,화,삼성,넥센,대구,24000


In [132]:
df_baseball_2018 = df_baseball_2018.head(-1)
#df_baseball_2018 = df_baseball_2018.iloc[:-1]
#df_baseball_2018 = df_baseball_2018.drop(len(df_baseball_2018)-1,axis=0)
df_baseball_2018.tail()

,날짜,요일,홈,방문,구장,관중수
715,2018/10/13,토,SK,LG,문학,22399
716,2018/10/13,토,KIA,롯데,광주,15640
717,2018/10/13,토,한화,NC,대전,13000
718,2018/10/13,토,삼성,넥센,대구,24000
719,2018/10/14,일,롯데,두산,사직,6414


#### <실습>
 - 각 날씨 데이터의 첫 3줄과 마지막 3줄을 출력해보자

In [133]:
for each_df in df_weather_list:
    print(each_df.head(3), '\n')
    print(each_df.tail(3), '\n')

    지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
0  423  2017-01-01 00:00     0.2     35.3      0.6      0.0        NaN   
1  423  2017-01-01 01:00     0.2      8.4      0.5      0.0        NaN   
2  423  2017-01-01 02:00    -0.3      0.0      0.1      0.0        NaN   

   해면기압(hPa)  습도(%)  일사(MJ/m^2)  일조(hr)  
0        NaN   91.5           0       0  
1        NaN   91.7           0       0  
2        NaN   92.9           0       0   

       지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
8757  423  2017-12-31 21:00    -0.7    326.6      1.9      0.0        NaN   
8758  423  2017-12-31 22:00    -1.2    344.9      0.6      0.0        NaN   
8759  423  2017-12-31 23:00    -1.7      0.0      0.2      0.0        NaN   

      해면기압(hPa)  습도(%)  일사(MJ/m^2)  일조(hr)  
8757        NaN   40.0           0       0  
8758        NaN   43.6           0       0  
8759        NaN   49.6           0       0   

    지점                일시  기온(°C)  풍향(deg) 

2        NaN    NaN           0       0   

       지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
8756  940  2017-12-31 21:00     3.8    310.1      1.5      0.0        NaN   
8757  940  2017-12-31 22:00     2.3    331.9      0.6      0.0        NaN   
8758  940  2017-12-31 23:00     1.8    343.0      1.2      0.0        NaN   

      해면기압(hPa)  습도(%)  일사(MJ/m^2)  일조(hr)  
8756        NaN   26.7           0       0  
8757        NaN   34.1           0       0  
8758        NaN   31.8           0       0   

    지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
0  403  2017-01-01 00:00     2.9    160.1      1.2      0.0        NaN   
1  403  2017-01-01 01:00     2.2    172.3      1.3      0.0        NaN   
2  403  2017-01-01 02:00     1.7    154.8      1.1      0.0        NaN   

   해면기압(hPa)  습도(%)  일사(MJ/m^2)  일조(hr)  
0        NaN    NaN           0       0  
1        NaN    NaN           0       0  
2        NaN    NaN           0       0   


### DataFrame 병합하기


#### <실습>
 - df_baseball_2017 과 df_baseball_2018 의 야구 데이터와
 - df_weather_list 에 저장된 날씨 데이터들을 하나의 DataFrame으로 합쳐보자

In [134]:
df_baseball = pd.concat([df_baseball_2017, df_baseball_2018])
print(df_baseball)

df_baseball.loc[0]

             날짜 요일    홈   방문  구장    관중수
0    2017/03/31  금   두산   한화  잠실  21121
1    2017/03/31  금   삼성  KIA  대구  13505
2    2017/03/31  금   SK   KT  문학  13649
3    2017/03/31  금   넥센   LG  고척   8013
4    2017/03/31  금   NC   롯데  마산  11000
5    2017/04/01  토   두산   한화  잠실  20636
6    2017/04/01  토   삼성  KIA  대구  10062
7    2017/04/01  토   SK   KT  문학  11291
8    2017/04/01  토   넥센   LG  고척   8889
9    2017/04/01  토   NC   롯데  마산   9596
10   2017/04/02  일   두산   한화  잠실  22457
11   2017/04/02  일   삼성  KIA  대구  14406
12   2017/04/02  일   SK   KT  문학  10186
13   2017/04/02  일   넥센   LG  고척   9130
14   2017/04/02  일   NC   롯데  마산  11000
15   2017/04/04  화   한화   NC  대전  10597
16   2017/04/04  화   KT   두산  수원  15804
17   2017/04/04  화  KIA   SK  광주  18069
18   2017/04/04  화   LG   삼성  잠실  16411
19   2017/04/04  화   롯데   넥센  사직  24954
20   2017/04/06  목   한화   NC  대전   6150
21   2017/04/06  목   KT   두산  수원   4426
22   2017/04/06  목  KIA   SK  광주   4211
23   2017/04/06  목   LG   삼성  잠실  12166


,날짜,요일,홈,방문,구장,관중수
0,2017/03/31,금,두산,한화,잠실,21121
0,2018/03/24,토,넥센,한화,고척,15055


 - Index가 중복된다!! ignore_index 전달인자를 사용

In [135]:
df_baseball = pd.concat([df_baseball_2017, df_baseball_2018], ignore_index=True)
print(df_baseball)

              날짜 요일    홈   방문  구장    관중수
0     2017/03/31  금   두산   한화  잠실  21121
1     2017/03/31  금   삼성  KIA  대구  13505
2     2017/03/31  금   SK   KT  문학  13649
3     2017/03/31  금   넥센   LG  고척   8013
4     2017/03/31  금   NC   롯데  마산  11000
5     2017/04/01  토   두산   한화  잠실  20636
6     2017/04/01  토   삼성  KIA  대구  10062
7     2017/04/01  토   SK   KT  문학  11291
8     2017/04/01  토   넥센   LG  고척   8889
9     2017/04/01  토   NC   롯데  마산   9596
10    2017/04/02  일   두산   한화  잠실  22457
11    2017/04/02  일   삼성  KIA  대구  14406
12    2017/04/02  일   SK   KT  문학  10186
13    2017/04/02  일   넥센   LG  고척   9130
14    2017/04/02  일   NC   롯데  마산  11000
15    2017/04/04  화   한화   NC  대전  10597
16    2017/04/04  화   KT   두산  수원  15804
17    2017/04/04  화  KIA   SK  광주  18069
18    2017/04/04  화   LG   삼성  잠실  16411
19    2017/04/04  화   롯데   넥센  사직  24954
20    2017/04/06  목   한화   NC  대전   6150
21    2017/04/06  목   KT   두산  수원   4426
22    2017/04/06  목  KIA   SK  광주   4211
23    2017/04/06

In [136]:
df_weather = pd.concat(df_weather_list, ignore_index=True)
print(df_weather)

         지점                일시  기온(°C)  풍향(deg)  풍속(m/s)  강수량(mm)  현지기압(hPa)  \
0       423  2017-01-01 00:00     0.2     35.3      0.6      0.0        NaN   
1       423  2017-01-01 01:00     0.2      8.4      0.5      0.0        NaN   
2       423  2017-01-01 02:00    -0.3      0.0      0.1      0.0        NaN   
3       423  2017-01-01 03:00    -0.4      0.0      0.0      0.0        NaN   
4       423  2017-01-01 04:00    -0.7      0.0      0.2      0.0        NaN   
5       423  2017-01-01 05:00    -0.6      0.0      0.0      0.0        NaN   
6       423  2017-01-01 06:00    -1.1    350.1      0.5      0.0        NaN   
7       423  2017-01-01 07:00    -0.4      0.0      0.0      0.0        NaN   
8       423  2017-01-01 08:00    -0.8    219.7      0.6      0.0        NaN   
9       423  2017-01-01 09:00     0.9     91.4      0.7      0.0        NaN   
10      423  2017-01-01 10:00     2.3      0.0      0.2      0.0        NaN   
11      423  2017-01-01 11:00     4.2    119.5      

### 추가적인 데이터 처리

#### <실습>
 - 날씨 데이터가 어떤 구장에 해당하는 날씨를 나타내는지 알기가 힘들다
 - 날씨 데이터의 지점코드에 대응하는 '구장' 컬럼을 추가해보자
 - 지점코드에 상응하는 구장명은 코드에 제공

In [137]:
df_weather.tail()

,지점,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),현지기압(hPa),해면기압(hPa),습도(%),일사(MJ/m^2),일조(hr)
174689,403,2017-12-31 19:00,1.6,305.8,2.3,0.0,NaN,NaN,NaN,0.0,0.0
174690,403,2017-12-31 20:00,1.0,302.1,2.6,0.0,NaN,NaN,NaN,0.0,0.0
174691,403,2017-12-31 21:00,0.5,300.6,2.9,0.0,NaN,NaN,NaN,0.0,0.0
174692,403,2017-12-31 22:00,0.3,301.5,2.6,0.0,NaN,NaN,NaN,0.0,0.0
174693,403,2017-12-31 23:00,0.0,296.7,3.1,0.0,NaN,NaN,NaN,0.0,0.0


In [138]:
code_info = {
    423 : '고척',
    512 : '문학',
    403 : '잠실',
    788 : '광주',
    642 : '대전',
    940 : '사직',
    550 : '수원',
    926 : '창원',
    327 : '청주',
    830 : '포항'
}

df_weather['구장'] = df_weather.apply(lambda row: code_info[row['지점']], axis=1)
df_weather.tail()

,지점,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),현지기압(hPa),해면기압(hPa),습도(%),일사(MJ/m^2),일조(hr),구장
174689,403,2017-12-31 19:00,1.6,305.8,2.3,0.0,NaN,NaN,NaN,0.0,0.0,잠실
174690,403,2017-12-31 20:00,1.0,302.1,2.6,0.0,NaN,NaN,NaN,0.0,0.0,잠실
174691,403,2017-12-31 21:00,0.5,300.6,2.9,0.0,NaN,NaN,NaN,0.0,0.0,잠실
174692,403,2017-12-31 22:00,0.3,301.5,2.6,0.0,NaN,NaN,NaN,0.0,0.0,잠실
174693,403,2017-12-31 23:00,0.0,296.7,3.1,0.0,NaN,NaN,NaN,0.0,0.0,잠실


#### <실습>
 - 아무래도 현지기압, 해면기압, 습도, 일사, 일조에 대한 데이터는 사용하지 않을 듯 하다
 - 필요 없는 '현지기압', '해면기압', '습도', '일사', '일조' 컬럼을 삭제해보자
 - 또한 나머지 컬럼명에서 단위표시를 삭제해보자 (예 : 풍향(deg) -> 풍향)

In [139]:
df_weather = df_weather.drop(['현지기압(hPa)', '해면기압(hPa)','습도(%)','일사(MJ/m^2)','일조(hr)'],axis=1)
print(df_weather)

In [143]:
df_weather = df_weather.rename(columns=lambda colname: colname.split('(')[0])
print(df_weather)

         지점                일시   기온     풍향   풍속  강수량  구장
0       423  2017-01-01 00:00  0.2   35.3  0.6  0.0  고척
1       423  2017-01-01 01:00  0.2    8.4  0.5  0.0  고척
2       423  2017-01-01 02:00 -0.3    0.0  0.1  0.0  고척
3       423  2017-01-01 03:00 -0.4    0.0  0.0  0.0  고척
4       423  2017-01-01 04:00 -0.7    0.0  0.2  0.0  고척
5       423  2017-01-01 05:00 -0.6    0.0  0.0  0.0  고척
6       423  2017-01-01 06:00 -1.1  350.1  0.5  0.0  고척
7       423  2017-01-01 07:00 -0.4    0.0  0.0  0.0  고척
8       423  2017-01-01 08:00 -0.8  219.7  0.6  0.0  고척
9       423  2017-01-01 09:00  0.9   91.4  0.7  0.0  고척
10      423  2017-01-01 10:00  2.3    0.0  0.2  0.0  고척
11      423  2017-01-01 11:00  4.2  119.5  1.0  0.0  고척
12      423  2017-01-01 12:00  6.4  168.3  1.2  0.0  고척
13      423  2017-01-01 13:00  7.2  200.8  2.7  0.0  고척
14      423  2017-01-01 14:00  8.0  209.4  2.1  0.0  고척
15      423  2017-01-01 15:00  7.3  182.3  1.6  0.0  고척
16      423  2017-01-01 16:00  7.5  205.4  1.8  

### 전처리가 끝난 데이터의 모습 다시 확인하기

In [144]:
df_weather.head(10)

,지점,일시,기온,풍향,풍속,강수량,구장
0,423,2017-01-01 00:00,0.2,35.3,0.6,0.0,고척
1,423,2017-01-01 01:00,0.2,8.4,0.5,0.0,고척
2,423,2017-01-01 02:00,-0.3,0.0,0.1,0.0,고척
3,423,2017-01-01 03:00,-0.4,0.0,0.0,0.0,고척
4,423,2017-01-01 04:00,-0.7,0.0,0.2,0.0,고척
5,423,2017-01-01 05:00,-0.6,0.0,0.0,0.0,고척
6,423,2017-01-01 06:00,-1.1,350.1,0.5,0.0,고척
7,423,2017-01-01 07:00,-0.4,0.0,0.0,0.0,고척
8,423,2017-01-01 08:00,-0.8,219.7,0.6,0.0,고척
9,423,2017-01-01 09:00,0.9,91.4,0.7,0.0,고척


In [145]:
df_weather.tail(10)

,지점,일시,기온,풍향,풍속,강수량,구장
174684,403,2017-12-31 14:00,2.8,300.6,3.7,0.0,잠실
174685,403,2017-12-31 15:00,3.1,298.5,3.6,0.0,잠실
174686,403,2017-12-31 16:00,2.9,295.3,3.0,0.0,잠실
174687,403,2017-12-31 17:00,2.3,305.4,4.4,0.0,잠실
174688,403,2017-12-31 18:00,2.0,300.7,3.6,0.0,잠실
174689,403,2017-12-31 19:00,1.6,305.8,2.3,0.0,잠실
174690,403,2017-12-31 20:00,1.0,302.1,2.6,0.0,잠실
174691,403,2017-12-31 21:00,0.5,300.6,2.9,0.0,잠실
174692,403,2017-12-31 22:00,0.3,301.5,2.6,0.0,잠실
174693,403,2017-12-31 23:00,0.0,296.7,3.1,0.0,잠실


In [146]:
df_baseball.head(10)

,날짜,요일,홈,방문,구장,관중수
0,2017/03/31,금,두산,한화,잠실,21121
1,2017/03/31,금,삼성,KIA,대구,13505
2,2017/03/31,금,SK,KT,문학,13649
3,2017/03/31,금,넥센,LG,고척,8013
4,2017/03/31,금,NC,롯데,마산,11000
5,2017/04/01,토,두산,한화,잠실,20636
6,2017/04/01,토,삼성,KIA,대구,10062
7,2017/04/01,토,SK,KT,문학,11291
8,2017/04/01,토,넥센,LG,고척,8889
9,2017/04/01,토,NC,롯데,마산,9596


In [147]:
df_baseball.tail(10)

,날짜,요일,홈,방문,구장,관중수
1430,2018/10/11,목,두산,SK,잠실,10987
1431,2018/10/12,금,KIA,롯데,광주,12318
1432,2018/10/12,금,두산,NC,잠실,10914
1433,2018/10/12,금,KT,넥센,수원,10268
1434,2018/10/13,토,두산,KT,잠실,18528
1435,2018/10/13,토,SK,LG,문학,22399
1436,2018/10/13,토,KIA,롯데,광주,15640
1437,2018/10/13,토,한화,NC,대전,13000
1438,2018/10/13,토,삼성,넥센,대구,24000
1439,2018/10/14,일,롯데,두산,사직,6414


### 각 DataFrame을 파일로 저장

In [148]:
df_baseball.to_csv("data/baseball.csv")
df_weather.to_csv("data/weather.csv")